In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Thư viện**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import argparse
import os
import sys
import math

from sklearn import metrics
from sklearn.metrics import precision_score

from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

from tensorflow.keras.metrics import Accuracy

from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.layers import RepeatVector
from keras.layers import Dropout

from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, TensorBoard, CSVLogger, EarlyStopping

import tensorflow as tf
from tensorflow.compat.v1.keras.backend import set_session
 

# **Import code**

In [ ]:
from google.colab import files
src = list(files.upload().values())[0]

open('augmentation.py','wb').write(src)
import augmentation as augmentation

Saving augmentation.py to augmentation.py


# **Import data**

In [ ]:
df_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Subject/Chuỗi thời gian/Cuối kỳ/CBF_TEST.csv')

df_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Subject/Chuỗi thời gian/Cuối kỳ/CBF_TRAIN.csv')

In [ ]:
df_train

,Label,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16,A17,A18,A19,A20,A21,A22,A23,A24,A25,A26,A27,A28,A29,A30,A31,A32,A33,A34,A35,A36,A37,A38,A39,...,A89,A90,A91,A92,A93,A94,A95,A96,A97,A98,A99,A100,A101,A102,A103,A104,A105,A106,A107,A108,A109,A110,A111,A112,A113,A114,A115,A116,A117,A118,A119,A120,A121,A122,A123,A124,A125,A126,A127,A128
0,1,-0.464276,-0.555048,-0.842843,-0.865895,-0.936396,-0.817270,-0.263612,-1.258048,-1.250393,-0.918308,-0.922102,-0.984488,-1.288051,-1.143460,-1.048861,-0.365386,-0.691395,-0.980550,-0.991336,-0.887087,-1.109881,-0.768725,-1.010296,-0.846996,-1.327746,-0.712909,-1.297727,1.331464,0.935738,1.236047,1.056068,0.910745,1.371348,0.937059,0.910292,0.931087,1.493780,1.319266,1.219089,...,-0.566059,-0.648499,-0.656493,-0.532251,-0.115048,-0.573152,-0.541183,-0.700087,-1.198927,-0.952699,-0.998282,-0.824990,-0.380780,-0.476598,-0.601361,-0.369363,-1.299045,-0.918395,-1.264813,-0.991938,-0.805577,-0.920951,-0.366233,-0.346363,-1.046916,-0.914667,-0.911570,-0.986775,-1.218954,-0.529620,-1.331086,-1.083700,-1.017961,-0.916029,-1.134289,-0.920224,-0.789362,-0.638706,-0.963664,-1.245169
1,1,-0.896972,-0.685686,-1.351382,-1.458667,-1.165346,-1.403929,-1.821800,-0.831601,-1.016312,-0.803530,-1.259505,-1.139234,-0.878652,-1.548725,-0.924158,-1.395482,0.776609,1.100727,1.189804,0.375148,0.947188,0.803713,1.157285,0.886792,1.037152,0.815015,0.790840,0.541211,0.858331,0.673374,0.444587,-0.121831,0.919671,0.894275,0.806385,0.511277,1.344212,0.527793,0.879901,...,1.058562,1.050804,0.260998,0.525592,0.745429,1.273054,-1.438041,-1.047012,-1.286655,-1.482158,-1.530711,-0.785110,-0.960946,-1.187367,-1.278416,-1.604283,-1.070981,-1.675471,-1.365958,-0.759499,-1.235802,-0.924245,-0.423050,-0.794589,-0.562493,-1.149179,-1.581894,-1.776748,-1.081367,-1.733105,-1.062553,-1.568676,-1.364040,-1.155177,-0.995861,-1.163753,-0.916523,-0.582098,-1.259009,-1.392524
2,1,-0.464696,-0.567739,-0.032023,-0.635046,-0.602826,-0.266856,-0.267061,-0.931042,-0.449382,-0.721342,-0.397272,-0.962126,-1.420667,-0.783434,-0.462386,-1.176224,-0.501638,-0.584869,-0.579163,-0.921298,-0.716378,-0.547787,-0.990844,-0.146466,-0.720035,1.158976,0.676715,1.428669,1.445366,1.320558,1.435816,1.693966,1.480389,1.402804,1.801457,1.581473,1.537731,1.330378,1.016200,...,-0.954912,-0.835666,-0.068535,-0.996553,-0.575302,-0.878510,-0.573001,-1.312447,-0.764347,-1.298105,-0.665513,-1.134686,-0.756020,-0.201630,-0.523471,-0.547361,-0.649560,-0.502961,-1.048601,-0.427315,-0.528524,-0.372422,-0.535811,-1.067065,-0.275338,-1.261341,-0.644883,-0.647301,-0.619081,-1.555553,-0.286721,-0.637158,-0.155260,-0.688129,-0.885609,-0.766239,-0.865315,-0.284486,-0.687889,-0.887608
3,3,-0.187190,-0.620808,-0.815661,-0.521398,-0.790423,-0.967517,-1.487006,-0.277887,-0.835075,0.050591,-0.811168,-1.226008,-0.668360,-0.776681,0.064062,-1.253650,-0.788991,-0.934222,-1.093595,-0.607904,-0.702728,0.018323,2.975575,2.677284,2.395800,1.421509,1.977175,2.577764,1.797997,2.080978,2.428340,2.055551,1.896724,1.543242,1.444597,1.811606,1.995693,1.282453,1.162219,...,-0.640391,0.054909,-1.061023,-0.114513,-0.428446,-0.226867,-0.541122,-0.539731,-0.140832,-1.052845,-0.031282,-0.291942,-0.331215,-0.845297,-0.323013,-1.770715,-0.352471,-0.805470,-1.103600,0.043254,-0.149619,-0.116496,-0.301638,0.684544,-0.869741,-0.232569,-0.099145,-0.626641,-0.675945,-0.131149,-0.908615,-0.726286,0.183778,-0.737444,-1.113997,-0.393987,-0.587889,-0.608232,-0.636465,-0.349029
4,2,-1.136017,-1.319195,-1.844624,-0.788692,-0.251715,-1.487603,-0.668764,-0.340360,-1.046382,-0.312214,-1.031442,-1.200578,-0.650476,-1.619584,-0.412751,-1.050671,-1.175124,-0.427838,-1.200284,-0.587265,-1.173467,-0.318766,-1.709967,-1.691622,-1.054783,-1.331545,-0.689782,-0.744991,-0.983910,-1.191218,-0.549003,0.584918,-0.592284,-0.762793,-0.282961,-0.435384,0.126199,-0.328959,-1.454095,...,0.627666,0.804837,1.257626,0.278243,1.011659,0.948171,1.313149,1.056891,0.868027,0.672851,1.169901,2.153332,1.730302,1.775781,1.313152,1.721768

**Chia dữ liệu**

In [ ]:
features = df_train.columns[1:]

In [ ]:
df_train_X = df_train[features]
df_test_X = df_test[features]

y_train = df_train['Label'].values
y_test = df_test['Label'].values

In [ ]:
nb_dims = 1
x_train = df_train_X.values
x_test = df_test_X.values

nb_timesteps = int(x_train.shape[1] / nb_dims)
input_shape = (nb_timesteps , nb_dims)

x_train_max = np.max(x_train)
x_train_min = np.min(x_train)
x_train = 2. * (x_train - x_train_min) / (x_train_max - x_train_min) - 1.
# Test is secret
x_test = 2. * (x_test - x_train_min) / (x_train_max - x_train_min) - 1.

x_test = x_test.reshape((-1, input_shape[0], input_shape[1])) 
x_train = x_train.reshape((-1, input_shape[0], input_shape[1])) 

In [ ]:
x_train

array([[[-0.33377329],
        [-0.36641649],
        [-0.46991339],
        ...,
        [-0.39650156],
        [-0.51336308],
        [-0.61459784]],

       [[-0.48937925],
        [-0.41339639],
        [-0.65279404],
        ...,
        [-0.37614427],
        [-0.61957508],
        [-0.66758963]],

       [[-0.33392414],
        [-0.37098044],
        [-0.17832623],
        ...,
        [-0.26911694],
        [-0.41418885],
        [-0.48601186]],

       ...,

       [[-0.23171899],
        [-0.52244956],
        [-0.27530829],
        ...,
        [-0.15970429],
        [-0.50561979],
        [-0.16209173]],

       [[-0.26242889],
        [-0.16208914],
        [-0.53680928],
        ...,
        [-0.31265203],
        [-0.50148404],
        [-0.44634377]],

       [[-0.52402617],
        [-0.47947077],
        [-0.29092775],
        ...,
        [-0.58787498],
        [-0.64334596],
        [-0.48851986]]])

**Đưa dữ liệu 3D về 2D**

In [ ]:
# x_test = x_test.reshape(300,128)
x_test = x_test.reshape(900,128)

**Y_new**

In [ ]:
# 10 
y_train_new = y_train

for i in range(0,99):
  y_train_new = np.append(y_train_new, y_train)

In [ ]:
y_train_new.shape

(3000,)

# **12 Phương pháp sinh dữ liệu**

**Hàm bổ trợ**

In [ ]:
import numpy as np
import math
import sys

RETURN_VALUE = 0
RETURN_PATH = 1
RETURN_ALL = -1

# Core DTW
def _traceback(DTW, slope_constraint):
    i, j = np.array(DTW.shape) - 1
    p, q = [i-1], [j-1]
    
    if slope_constraint == "asymmetric":
        while (i > 1):
            tb = np.argmin((DTW[i-1, j], DTW[i-1, j-1], DTW[i-1, j-2]))

            if (tb == 0):
                i = i - 1
            elif (tb == 1):
                i = i - 1
                j = j - 1
            elif (tb == 2):
                i = i - 1
                j = j - 2

            p.insert(0, i-1)
            q.insert(0, j-1)
    elif slope_constraint == "symmetric":
        while (i > 1 or j > 1):
            tb = np.argmin((DTW[i-1, j-1], DTW[i-1, j], DTW[i, j-1]))

            if (tb == 0):
                i = i - 1
                j = j - 1
            elif (tb == 1):
                i = i - 1
            elif (tb == 2):
                j = j - 1

            p.insert(0, i-1)
            q.insert(0, j-1)
    else:
        sys.exit("Unknown slope constraint %s"%slope_constraint)
        
    return (np.array(p), np.array(q))

def _cummulative_matrix(cost, slope_constraint, window):
    p = cost.shape[0]
    s = cost.shape[1]
    
    # Note: DTW is one larger than cost and the original patterns
    DTW = np.full((p+1, s+1), np.inf)

    DTW[0, 0] = 0.0

    if slope_constraint == "asymmetric":
        for i in range(1, p+1):
            if i <= window+1:
                DTW[i,1] = cost[i-1,0] + min(DTW[i-1,0], DTW[i-1,1])
            for j in range(max(2, i-window), min(s, i+window)+1):
                DTW[i,j] = cost[i-1,j-1] + min(DTW[i-1,j-2], DTW[i-1,j-1], DTW[i-1,j])
    elif slope_constraint == "symmetric":
        for i in range(1, p+1):
            for j in range(max(1, i-window), min(s, i+window)+1):
                DTW[i,j] = cost[i-1,j-1] + min(DTW[i-1,j-1], DTW[i,j-1], DTW[i-1,j])
    else:
        sys.exit("Unknown slope constraint %s"%slope_constraint)
        
    return DTW

def shape_dtw(prototype, sample, return_flag = RETURN_VALUE, slope_constraint="asymmetric", window=None, descr_ratio=0.05):
    """ Computes the shapeDTW of two sequences.
    :param prototype: np array [0..b]
    :param sample: np array [0..t]
    :param extended: bool
    """
    # shapeDTW
    # https://www.sciencedirect.com/science/article/pii/S0031320317303710
    
    p = prototype.shape[0]
    assert p != 0, "Prototype empty!"
    s = sample.shape[0]
    assert s != 0, "Sample empty!"
    
    if window is None:
        window = s
        
    p_feature_len = np.clip(np.round(p * descr_ratio), 5, 100).astype(int)
    s_feature_len = np.clip(np.round(s * descr_ratio), 5, 100).astype(int)
    
    # padding
    p_pad_front = (np.ceil(p_feature_len / 2.)).astype(int)
    p_pad_back = (np.floor(p_feature_len / 2.)).astype(int)
    s_pad_front = (np.ceil(s_feature_len / 2.)).astype(int)
    s_pad_back = (np.floor(s_feature_len / 2.)).astype(int)
    
    prototype_pad = np.pad(prototype, ((p_pad_front, p_pad_back), (0, 0)), mode="edge") 
    sample_pad = np.pad(sample, ((s_pad_front, s_pad_back), (0, 0)), mode="edge") 
    p_p = prototype_pad.shape[0]
    s_p = sample_pad.shape[0]
        
    cost = np.full((p, s), np.inf)
    for i in range(p):
        for j in range(max(0, i-window), min(s, i+window)):
            cost[i, j] = np.linalg.norm(sample_pad[j:j+s_feature_len] - prototype_pad[i:i+p_feature_len])
            
    DTW = _cummulative_matrix(cost, slope_constraint=slope_constraint, window=window)
    
    if return_flag == RETURN_ALL:
        return DTW[-1,-1], cost, DTW[1:,1:], _traceback(DTW, slope_constraint)
    elif return_flag == RETURN_PATH:
        return _traceback(DTW, slope_constraint)
    else:
        return DTW[-1,-1]
    
# Draw helpers
def draw_graph2d(cost, DTW, path, prototype, sample):
    import matplotlib.pyplot as plt
    plt.figure(figsize=(12, 8))
   # plt.subplots_adjust(left=.02, right=.98, bottom=.001, top=.96, wspace=.05, hspace=.01)

    #cost
    plt.subplot(2, 3, 1)
    plt.imshow(cost.T, cmap=plt.cm.gray, interpolation='none', origin='lower')
    plt.plot(path[0], path[1], 'y')
    plt.xlim((-0.5, cost.shape[0]-0.5))
    plt.ylim((-0.5, cost.shape[0]-0.5))

    #dtw
    plt.subplot(2, 3, 2)
    plt.imshow(DTW.T, cmap=plt.cm.gray, interpolation='none', origin='lower')
    plt.plot(path[0]+1, path[1]+1, 'y')
    plt.xlim((-0.5, DTW.shape[0]-0.5))
    plt.ylim((-0.5, DTW.shape[0]-0.5))

    #prototype
    plt.subplot(2, 3, 4)
    plt.plot(prototype[:,0], prototype[:,1], 'b-o')

    #connection
    plt.subplot(2, 3, 5)
    for i in range(0,path[0].shape[0]):
        plt.plot([prototype[path[0][i],0], sample[path[1][i],0]],[prototype[path[0][i],1], sample[path[1][i],1]], 'y-')
    plt.plot(sample[:,0], sample[:,1], 'g-o')
    plt.plot(prototype[:,0], prototype[:,1], 'b-o')

    #sample
    plt.subplot(2, 3, 6)
    plt.plot(sample[:,0], sample[:,1], 'g-o')

    plt.tight_layout()
    plt.show()

def draw_graph1d(cost, DTW, path, prototype, sample):
    import matplotlib.pyplot as plt
    plt.figure(figsize=(12, 8))
   # plt.subplots_adjust(left=.02, right=.98, bottom=.001, top=.96, wspace=.05, hspace=.01)
    p_steps = np.arange(prototype.shape[0])
    s_steps = np.arange(sample.shape[0])

    #cost
    plt.subplot(2, 3, 1)
    plt.imshow(cost.T, cmap=plt.cm.gray, interpolation='none', origin='lower')
    plt.plot(path[0], path[1], 'y')
    plt.xlim((-0.5, cost.shape[0]-0.5))
    plt.ylim((-0.5, cost.shape[0]-0.5))

    #dtw
    plt.subplot(2, 3, 2)
    plt.imshow(DTW.T, cmap=plt.cm.gray, interpolation='none', origin='lower')
    plt.plot(path[0]+1, path[1]+1, 'y')
    plt.xlim((-0.5, DTW.shape[0]-0.5))
    plt.ylim((-0.5, DTW.shape[0]-0.5))

    #prototype
    plt.subplot(2, 3, 4)
    plt.plot(p_steps, prototype[:,0], 'b-o')

    #connection
    plt.subplot(2, 3, 5)
    for i in range(0,path[0].shape[0]):
        plt.plot([path[0][i], path[1][i]],[prototype[path[0][i],0], sample[path[1][i],0]], 'y-')
    plt.plot(p_steps, sample[:,0], 'g-o')
    plt.plot(s_steps, prototype[:,0], 'b-o')

    #sample
    plt.subplot(2, 3, 6)
    plt.plot(s_steps, sample[:,0], 'g-o')

    plt.tight_layout()
    plt.show()

In [ ]:
def dtw(prototype, sample, return_flag = 0, slope_constraint="asymmetric", window=None):
    """ Computes the DTW of two sequences.
    :param prototype: np array [0..b]
    :param sample: np array [0..t]
    :param extended: bool
    """
    p = prototype.shape[0]
    assert p != 0, "Prototype empty!"
    s = sample.shape[0]
    assert s != 0, "Sample empty!"
    
    if window is None:
        window = s
    
    cost = np.full((p, s), np.inf)
    for i in range(p):
        start = max(0, i-window)
        end = min(s, i+window)+1
        cost[i,start:end]=np.linalg.norm(sample[start:end] - prototype[i], axis=1)

    DTW = _cummulative_matrix(cost, slope_constraint, window)
        
    if return_flag == RETURN_ALL:
        return DTW[-1,-1], cost, DTW[1:,1:], _traceback(DTW, slope_constraint)
    elif return_flag == RETURN_PATH:
        return _traceback(DTW, slope_constraint)
    else:
        return DTW[-1,-1]

**spawner**

In [ ]:
def spawner(x, labels, sigma=0.05, verbose=0):
    # https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6983028/
    # use verbose=-1 to turn off warnings
    # use verbose=1 to print out figures
    
   # import dtw as dtw
    random_points = np.random.randint(low=1, high=x.shape[1]-1, size=x.shape[0])
    window = np.ceil(x.shape[1] / 10.).astype(int)
    orig_steps = np.arange(x.shape[1])
    l = np.argmax(labels, axis=1) if labels.ndim > 1 else labels
    
    ret = np.zeros_like(x)
    for i, pat in enumerate(tqdm(x)):
        # guarentees that same one isnt selected
        choices = np.delete(np.arange(x.shape[0]), i)
        # remove ones of different classes
        choices = np.where(l[choices] == l[i])[0]
        if choices.size > 0:     
            random_sample = x[np.random.choice(choices)]
            # SPAWNER splits the path into two randomly
            path1 = dtw(pat[:random_points[i]], random_sample[:random_points[i]], RETURN_PATH, slope_constraint="symmetric", window=window)
            path2 = dtw(pat[random_points[i]:], random_sample[random_points[i]:], RETURN_PATH, slope_constraint="symmetric", window=window)
            combined = np.concatenate((np.vstack(path1), np.vstack(path2+random_points[i])), axis=1)
            if verbose:
                print(random_points[i])
                dtw_value, cost, DTW_map, path = dtw(pat, random_sample, return_flag = RETURN_ALL, slope_constraint="symmetric", window=window)
                draw_graph1d(cost, DTW_map, path, pat, random_sample)
                draw_graph1d(cost, DTW_map, combined, pat, random_sample)
            mean = np.mean([pat[combined[0]], random_sample[combined[1]]], axis=0)
            for dim in range(x.shape[2]):
                ret[i,:,dim] = np.interp(orig_steps, np.linspace(0, x.shape[1]-1., num=mean.shape[0]), mean[:,dim]).T
        else:
            if verbose > -1:
                print("There is only one pattern of class %d, skipping pattern average"%l[i])
            ret[i,:] = pat
    return augmentation.jitter(ret, sigma=sigma)

**wdba**

In [ ]:
def wdba(x, labels, batch_size=6, slope_constraint="symmetric", use_window=True, verbose=0):
    # https://ieeexplore.ieee.org/document/8215569
    # use verbose = -1 to turn off warnings    
    # slope_constraint is for DTW. "symmetric" or "asymmetric"
    
    if use_window:
        window = np.ceil(x.shape[1] / 10.).astype(int)
    else:
        window = None
    orig_steps = np.arange(x.shape[1])
    l = np.argmax(labels, axis=1) if labels.ndim > 1 else labels
        
    ret = np.zeros_like(x)
    for i in tqdm(range(ret.shape[0])):
        # get the same class as i
        choices = np.where(l == l[i])[0]
        if choices.size > 0:        
            # pick random intra-class pattern
            k = min(choices.size, batch_size)
            random_prototypes = x[np.random.choice(choices, k, replace=False)]
            
            # calculate dtw between all
            dtw_matrix = np.zeros((k, k))
            for p, prototype in enumerate(random_prototypes):
                for s, sample in enumerate(random_prototypes):
                    if p == s:
                        dtw_matrix[p, s] = 0.
                    else:
                        dtw_matrix[p, s] = dtw(prototype, sample, RETURN_VALUE, slope_constraint=slope_constraint, window=window)
                        
            # get medoid
            medoid_id = np.argsort(np.sum(dtw_matrix, axis=1))[0]
            nearest_order = np.argsort(dtw_matrix[medoid_id])
            medoid_pattern = random_prototypes[medoid_id]
            
            # start weighted DBA
            average_pattern = np.zeros_like(medoid_pattern)
            weighted_sums = np.zeros((medoid_pattern.shape[0]))
            for nid in nearest_order:
                if nid == medoid_id or dtw_matrix[medoid_id, nearest_order[1]] == 0.:
                    average_pattern += medoid_pattern 
                    weighted_sums += np.ones_like(weighted_sums) 
                else:
                    path = dtw(medoid_pattern, random_prototypes[nid], RETURN_PATH, slope_constraint=slope_constraint, window=window)
                    dtw_value = dtw_matrix[medoid_id, nid]
                    warped = random_prototypes[nid, path[1]]
                    weight = np.exp(np.log(0.5)*dtw_value/dtw_matrix[medoid_id, nearest_order[1]])
                    average_pattern[path[0]] += weight * warped
                    weighted_sums[path[0]] += weight 
            
            ret[i,:] = average_pattern / weighted_sums[:,np.newaxis]
        else:
            if verbose > -1:
                print("There is only one pattern of class %d, skipping pattern average"%l[i])
            ret[i,:] = x[i]
    return ret

**random_guided_warp**

In [ ]:
def random_guided_warp(x, labels, slope_constraint="symmetric", use_window=True, dtw_type="normal", verbose=0):
    # use verbose = -1 to turn off warnings
    # slope_constraint is for DTW. "symmetric" or "asymmetric"
    # dtw_type is for shapeDTW or DTW. "normal" or "shape"
    
    if use_window:
        window = np.ceil(x.shape[1] / 10.).astype(int)
    else:
        window = None
    orig_steps = np.arange(x.shape[1])
    l = np.argmax(labels, axis=1) if labels.ndim > 1 else labels
    
    ret = np.zeros_like(x)
    for i, pat in enumerate(tqdm(x)):
        # guarentees that same one isnt selected
        choices = np.delete(np.arange(x.shape[0]), i)
        # remove ones of different classes
        choices = np.where(l[choices] == l[i])[0]
        if choices.size > 0:        
            # pick random intra-class pattern
            random_prototype = x[np.random.choice(choices)]
            
            if dtw_type == "shape":
                path = shape_dtw(random_prototype, pat, RETURN_PATH, slope_constraint=slope_constraint, window=window)
            else:
                path = dtw(random_prototype, pat, RETURN_PATH, slope_constraint=slope_constraint, window=window)
                            
            # Time warp
            warped = pat[path[1]]
            for dim in range(x.shape[2]):
                ret[i,:,dim] = np.interp(orig_steps, np.linspace(0, x.shape[1]-1., num=warped.shape[0]), warped[:,dim]).T
        else:
            if verbose > -1:
                print("There is only one pattern of class %d, skipping timewarping"%l[i])
            ret[i,:] = pat
    return ret

**discriminative_guided_warp**

In [ ]:
def discriminative_guided_warp(x, labels, batch_size=6, slope_constraint="symmetric", use_window=True, dtw_type="normal", use_variable_slice=True, verbose=0):
    # use verbose = -1 to turn off warnings
    # slope_constraint is for DTW. "symmetric" or "asymmetric"
    # dtw_type is for shapeDTW or DTW. "normal" or "shape"
    
    if use_window:
        window = np.ceil(x.shape[1] / 10.).astype(int)
    else:
        window = None
    orig_steps = np.arange(x.shape[1])
    l = np.argmax(labels, axis=1) if labels.ndim > 1 else labels
    
    positive_batch = np.ceil(batch_size / 2).astype(int)
    negative_batch = np.floor(batch_size / 2).astype(int)
        
    ret = np.zeros_like(x)
    warp_amount = np.zeros(x.shape[0])
    for i, pat in enumerate(tqdm(x)):
        # guarentees that same one isnt selected
        choices = np.delete(np.arange(x.shape[0]), i)
        
        # remove ones of different classes
        positive = np.where(l[choices] == l[i])[0]
        negative = np.where(l[choices] != l[i])[0]
        
        if positive.size > 0 and negative.size > 0:
            pos_k = min(positive.size, positive_batch)
            neg_k = min(negative.size, negative_batch)
            positive_prototypes = x[np.random.choice(positive, pos_k, replace=False)]
            negative_prototypes = x[np.random.choice(negative, neg_k, replace=False)]
                        
            # vector embedding and nearest prototype in one
            pos_aves = np.zeros((pos_k))
            neg_aves = np.zeros((pos_k))
            if dtw_type == "shape":
                for p, pos_prot in enumerate(positive_prototypes):
                    for ps, pos_samp in enumerate(positive_prototypes):
                        if p != ps:
                            pos_aves[p] += (1./(pos_k-1.))*shape_dtw(pos_prot, pos_samp, RETURN_VALUE, slope_constraint=slope_constraint, window=window)
                    for ns, neg_samp in enumerate(negative_prototypes):
                        neg_aves[p] += (1./neg_k)*shape_dtw(pos_prot, neg_samp, RETURN_VALUE, slope_constraint=slope_constraint, window=window)
                selected_id = np.argmax(neg_aves - pos_aves)
                path = shape_dtw(positive_prototypes[selected_id], pat, RETURN_PATH, slope_constraint=slope_constraint, window=window)
            else:
                for p, pos_prot in enumerate(positive_prototypes):
                    for ps, pos_samp in enumerate(positive_prototypes):
                        if p != ps:
                            pos_aves[p] += (1./(pos_k-1.))*dtw(pos_prot, pos_samp, RETURN_VALUE, slope_constraint=slope_constraint, window=window)
                    for ns, neg_samp in enumerate(negative_prototypes):
                        neg_aves[p] += (1./neg_k)*dtw(pos_prot, neg_samp, RETURN_VALUE, slope_constraint=slope_constraint, window=window)
                selected_id = np.argmax(neg_aves - pos_aves)
                path = dtw(positive_prototypes[selected_id], pat, RETURN_PATH, slope_constraint=slope_constraint, window=window)
                   
            # Time warp
            warped = pat[path[1]]
            warp_path_interp = np.interp(orig_steps, np.linspace(0, x.shape[1]-1., num=warped.shape[0]), path[1])
            warp_amount[i] = np.sum(np.abs(orig_steps-warp_path_interp))
            for dim in range(x.shape[2]):
                ret[i,:,dim] = np.interp(orig_steps, np.linspace(0, x.shape[1]-1., num=warped.shape[0]), warped[:,dim]).T
        else:
            if verbose > -1:
                print("There is only one pattern of class %d"%l[i])
            ret[i,:] = pat
            warp_amount[i] = 0.
    if use_variable_slice:
        max_warp = np.max(warp_amount)
        if max_warp == 0:
            # unchanged
            ret = augmentation.window_slice(ret, reduce_ratio=0.9)
        else:
            for i, pat in enumerate(ret):
                # Variable Sllicing
                ret[i] = augmentation.window_slice(pat[np.newaxis,:,:], reduce_ratio=0.9+0.1*warp_amount[i]/max_warp)[0]
    return ret


## **Hàm áp dụng dữ liệu sinh từ 12 Phương pháp lần lượt vào mô hình MÁY HỌC**

In [ ]:
def method_apply(model, x_train, y_train, x_test, y_test): 

  # Khong ap dung Sinh du lieu
  model.fit(x_train.reshape(30,128), y_train)
  y_pred = model.predict(x_test)
  print("Accuracy - None:",metrics.accuracy_score(y_pred,y_test)) 

  # Ap dung PP jitter
  x_train_jitter = x_train
  for i in range(0, 99):
    tmp1 = augmentation.jitter(x_train)
    x_train_jitter = np.append(x_train_jitter, tmp1, axis = 0)
  model.fit(x_train_jitter.reshape(3000,128), y_train_new)
  y_pred = model.predict(x_test)
  print("Accuracy - jitter:",metrics.accuracy_score(y_pred,y_test))

  # Ap dung PP scaling
  x_train_scaling = x_train
  for i in range(0, 99):
    tmp1 = augmentation.scaling(x_train)
    x_train_scaling = np.append(x_train_scaling, tmp1, axis = 0)
  model.fit(x_train_scaling.reshape(3000,128), y_train_new)
  y_pred = model.predict(x_test)
  print("Accuracy - scaling:",metrics.accuracy_score(y_pred,y_test))

  # Ap dung PP rotation
  x_train_rotation = x_train
  for i in range(0, 99):
    tmp1 = augmentation.rotation(x_train)
    x_train_rotation = np.append(x_train_rotation, tmp1, axis = 0)
  model.fit(x_train_rotation.reshape(3000,128), y_train_new)
  y_pred = model.predict(x_test)
  print("Accuracy - rotation:",metrics.accuracy_score(y_pred,y_test))

  #Ap dung PP permutation
  x_train_permutation = x_train
  for i in range(0, 99):
    tmp1 = augmentation.permutation(x_train)
    x_train_permutation = np.append(x_train_permutation, tmp1, axis = 0)
  model.fit(x_train_permutation.reshape(3000,128), y_train_new)
  y_pred = model.predict(x_test)
  print("Accuracy - permutation:",metrics.accuracy_score(y_pred,y_test))

  # Ap dung PP magnitude_warp
  x_train_magnitude_warp = x_train
  for i in range(0, 99):
    tmp1 = augmentation.magnitude_warp(x_train)
    x_train_magnitude_warp = np.append(x_train_magnitude_warp, tmp1, axis = 0)
  model.fit(x_train_magnitude_warp.reshape(3000,128), y_train_new)
  y_pred = model.predict(x_test)
  print("Accuracy - magnitude_warp:",metrics.accuracy_score(y_pred,y_test))

  # Ap dung PP time_warp
  x_train_time_warp = x_train
  for i in range(0, 99):
    tmp1 = augmentation.time_warp(x_train)
    x_train_time_warp = np.append(x_train_time_warp, tmp1, axis = 0)
  model.fit(x_train_time_warp.reshape(3000,128), y_train_new)
  y_pred = model.predict(x_test)
  print("Accuracy - time_warp:",metrics.accuracy_score(y_pred,y_test))

  # Ap dung PP window_slice
  x_train_window_slice = x_train
  for i in range(0, 99):
    tmp1 = augmentation.window_slice(x_train)
    x_train_window_slice = np.append(x_train_window_slice, tmp1, axis = 0)
  model.fit(x_train_window_slice.reshape(3000,128), y_train_new)
  y_pred = model.predict(x_test)
  print("Accuracy - window_slice:",metrics.accuracy_score(y_pred,y_test))

  # Ap dung PP window_warp
  x_train_window_warp = x_train
  for i in range(0, 99):
    tmp1 = augmentation.window_warp(x_train)
    x_train_window_warp = np.append(x_train_window_warp, tmp1, axis = 0)
  model.fit(x_train_window_warp.reshape(3000,128), y_train_new)
  y_pred = model.predict(x_test)
  print("Accuracy - window_warp:",metrics.accuracy_score(y_pred,y_test))

  # Ap dung PP spawner
  x_train_spawner = x_train
  for i in range(0, 99):
    tmp1 = spawner(x_train, y_train)
    x_train_spawner = np.append(x_train_spawner, tmp1, axis = 0)
  model.fit(x_train_spawner.reshape(3000,128), y_train_new)
  y_pred = model.predict(x_test)
  print("Accuracy - spawner:",metrics.accuracy_score(y_pred,y_test))

  # Ap dung PP wdba
  x_train_wdba = x_train
  for i in range(0, 99):
    tmp1 = wdba(x_train, y_train)
    x_train_wdba = np.append(x_train_wdba, tmp1, axis = 0)
  model.fit(x_train_wdba.reshape(3000,128), y_train_new)
  y_pred = model.predict(x_test)
  print("Accuracy - wdba:",metrics.accuracy_score(y_pred,y_test))

  # Ap dung PP random_guided_warp
  x_train_random_guided_warp = x_train
  for i in range(0, 99):
    tmp1 = random_guided_warp(x_train, y_train)
    x_train_random_guided_warp = np.append(x_train_random_guided_warp, tmp1, axis = 0)
  model.fit(x_train_random_guided_warp.reshape(3000,128), y_train_new)
  y_pred = model.predict(x_test)
  print("Accuracy - random_guided_warp:",metrics.accuracy_score(y_pred,y_test))

  # Ap dung PP discriminative_guided_warp
  x_train_discriminative_guided_warp = x_train
  for i in range(0, 99):
    tmp1 = discriminative_guided_warp(x_train, y_train)
    x_train_discriminative_guided_warp = np.append(x_train_discriminative_guided_warp, tmp1, axis = 0)
  model.fit(x_train_discriminative_guided_warp.reshape(3000,128), y_train_new)
  y_pred = model.predict(x_test)
  print("Accuracy - discriminative_guided_warp:",metrics.accuracy_score(y_pred,y_test))

# **Các mô hình Machine Learning cơ bản**

**Logistic Regression**

In [ ]:
 logreg = LogisticRegression(fit_intercept=True, penalty='l2')
 method_apply(logreg, x_train, y_train, x_test, y_test)

Accuracy - None: 0.8533333333333334
Accuracy - jitter: 0.8544444444444445
Accuracy - scaling: 0.8433333333333334
Accuracy - rotation: 0.45666666666666667


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to t

Accuracy - permutation: 0.3377777777777778
Accuracy - magnitude_warp: 0.86


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


Accuracy - time_warp: 0.7477777777777778


...

100%|██████████| 30/30 [00:00<00:00, 101.64it/s]
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


Accuracy - random_guided_warp: 0.8855555555555555


100%|██████████| 30/30 [00:03<00:00,  8.29it/s]


Accuracy - discriminative_guided_warp: 0.9077777777777778


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


**Linear Discriminant Analysis**

In [ ]:
lda = LinearDiscriminantAnalysis()
method_apply(lda, x_train, y_train, x_test, y_test)

Accuracy - None: 0.84
Accuracy - jitter: 0.8688888888888889
Accuracy - scaling: 0.7944444444444444
Accuracy - rotation: 0.35777777777777775


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


Accuracy - permutation: 0.35888888888888887
Accuracy - magnitude_warp: 0.41444444444444445
Accuracy - time_warp: 0.78
Accuracy - window_slice: 0.9411111111111111
Accuracy - window_warp: 0.7833333333333333


100%|██████████| 30/30 [00:00<00:00, 110.87it/s]


Accuracy - spawner: 0.8377777777777777


...

Accuracy - wdba: 0.8388888888888889


100%|██████████| 30/30 [00:00<00:00, 103.90it/s]


Accuracy - random_guided_warp: 0.7811111111111111


100%|██████████| 30/30 [00:03<00:00,  8.17it/s]


Accuracy - discriminative_guided_warp: 0.8866666666666667


**Gaussian Naive Bayes**

In [ ]:
gnb = GaussianNB()
method_apply(gnb, x_train, y_train, x_test, y_test)

Accuracy - None: 0.8944444444444445
Accuracy - jitter: 0.8966666666666666
Accuracy - scaling: 0.8988888888888888
Accuracy - rotation: 0.5777777777777777
Accuracy - permutation: 0.8611111111111112


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


Accuracy - magnitude_warp: 0.9077777777777778
Accuracy - time_warp: 0.8244444444444444
Accuracy - window_slice: 0.9088888888888889
Accuracy - window_warp: 0.8766666666666667


100%|██████████| 30/30 [00:00<00:00, 107.17it/s]


Accuracy - spawner: 0.8988888888888888


...

Accuracy - wdba: 0.7844444444444445


100%|██████████| 30/30 [00:00<00:00, 101.12it/s]


Accuracy - random_guided_warp: 0.9044444444444445


100%|██████████| 30/30 [00:03<00:00,  8.36it/s]

Accuracy - discriminative_guided_warp: 0.9033333333333333


**Decision Tree Classifier**

In [ ]:
dtc= DecisionTreeClassifier(random_state=1234)
method_apply(dtc, x_train, y_train, x_test, y_test)


Accuracy - None: 0.7566666666666667
Accuracy - jitter: 0.8011111111111111
Accuracy - scaling: 0.7166666666666667
Accuracy - rotation: 0.4722222222222222


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


Accuracy - permutation: 0.5411111111111111
Accuracy - magnitude_warp: 0.7311111111111112
Accuracy - time_warp: 0.7322222222222222
Accuracy - window_slice: 0.8255555555555556
Accuracy - window_warp: 0.8311111111111111


100%|██████████| 30/30 [00:00<00:00, 88.89it/s]


Accuracy - spawner: 0.7755555555555556


...

Accuracy - wdba: 0.7244444444444444


100%|██████████| 30/30 [00:00<00:00, 94.21it/s]


Accuracy - random_guided_warp: 0.7744444444444445


100%|██████████| 30/30 [00:03<00:00,  8.25it/s]


Accuracy - discriminative_guided_warp: 0.7855555555555556


**Random Forest Classifier**

In [ ]:
rfc=RandomForestClassifier(n_jobs=-1,random_state=123)
method_apply(rfc, x_train, y_train, x_test, y_test)

Accuracy - None: 0.8711111111111111
Accuracy - jitter: 0.9266666666666666
Accuracy - scaling: 0.9344444444444444
Accuracy - rotation: 0.7811111111111111


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


Accuracy - permutation: 0.8355555555555556
Accuracy - magnitude_warp: 0.9466666666666667
Accuracy - time_warp: 0.9544444444444444
Accuracy - window_slice: 0.9911111111111112
Accuracy - window_warp: 0.9511111111111111


100%|██████████| 30/30 [00:00<00:00, 90.35it/s]


Accuracy - spawner: 0.9488888888888889


...

Accuracy - wdba: 0.9055555555555556


100%|██████████| 30/30 [00:00<00:00, 86.97it/s]


Accuracy - random_guided_warp: 0.9677777777777777


100%|██████████| 30/30 [00:03<00:00,  7.51it/s]


Accuracy - discriminative_guided_warp: 0.9888888888888889


**K-NearestNeighbours Classifier**

In [ ]:
knn = KNeighborsClassifier()
method_apply(knn, x_train, y_train, x_test, y_test)

Accuracy - None: 0.7666666666666667
Accuracy - jitter: 0.8533333333333334
Accuracy - scaling: 0.85
Accuracy - rotation: 0.8522222222222222


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


Accuracy - permutation: 0.8377777777777777
Accuracy - magnitude_warp: 0.8477777777777777
Accuracy - time_warp: 0.9955555555555555
Accuracy - window_slice: 0.9733333333333334
Accuracy - window_warp: 0.9655555555555555


100%|██████████| 30/30 [00:00<00:00, 101.49it/s]


Accuracy - spawner: 0.9133333333333333


...

Accuracy - wdba: 0.8377777777777777


100%|██████████| 30/30 [00:00<00:00, 96.18it/s]


Accuracy - random_guided_warp: 0.93


100%|██████████| 30/30 [00:03<00:00,  8.41it/s]


Accuracy - discriminative_guided_warp: 0.9777777777777777


**Ada Boost**

In [ ]:
ada_model = AdaBoostClassifier(n_estimators=200, random_state=44)
method_apply(ada_model, x_train, y_train, x_test, y_test)


Accuracy - None: 0.81
Accuracy - jitter: 0.8066666666666666
Accuracy - scaling: 0.7333333333333333
Accuracy - rotation: 0.4855555555555556


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


Accuracy - permutation: 0.49333333333333335
Accuracy - magnitude_warp: 0.7933333333333333
Accuracy - time_warp: 0.6588888888888889
Accuracy - window_slice: 0.8133333333333334
Accuracy - window_warp: 0.8155555555555556


100%|██████████| 30/30 [00:00<00:00, 87.99it/s]


Accuracy - spawner: 0.9011111111111111


...

Accuracy - wdba: 0.7033333333333334


100%|██████████| 30/30 [00:00<00:00, 91.67it/s]


Accuracy - random_guided_warp: 0.8011111111111111


100%|██████████| 30/30 [00:03<00:00,  7.67it/s]


Accuracy - discriminative_guided_warp: 0.8922222222222222


**XGB Classifier**

In [ ]:
XGB = XGBClassifier(objective='binary:logistic',scale_pos_weight=1,n_estimators=85, eta=0.3, subsample=1, colsample_bytree=1)
method_apply(XGB, x_train, y_train, x_test, y_test)


Accuracy - None: 0.77
Accuracy - jitter: 0.8166666666666667
Accuracy - scaling: 0.8511111111111112
Accuracy - rotation: 0.5377777777777778


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


Accuracy - permutation: 0.7288888888888889
Accuracy - magnitude_warp: 0.8
Accuracy - time_warp: 0.7477777777777778
Accuracy - window_slice: 0.9355555555555556
Accuracy - window_warp: 0.8833333333333333


100%|██████████| 30/30 [00:00<00:00, 93.95it/s]


Accuracy - spawner: 0.9355555555555556


...

Accuracy - wdba: 0.7822222222222223


100%|██████████| 30/30 [00:00<00:00, 99.91it/s]


Accuracy - random_guided_warp: 0.8811111111111111


100%|██████████| 30/30 [00:03<00:00,  8.33it/s]


Accuracy - discriminative_guided_warp: 0.9466666666666667


**Light GBM**

In [ ]:
lgbm_clf = LGBMClassifier(n_estimators=1000, objective='binary',
                          class_weight={0:25, 1:1}, learning_rate=0.01,
                          boosting_type='dart')
method_apply(lgbm_clf, x_train, y_train, x_test, y_test)

Accuracy - None: 0.3333333333333333
Accuracy - jitter: 0.7177777777777777
Accuracy - scaling: 0.8066666666666666
Accuracy - rotation: 0.5977777777777777


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


Accuracy - permutation: 0.6822222222222222
Accuracy - magnitude_warp: 0.7544444444444445
Accuracy - time_warp: 0.8566666666666667
Accuracy - window_slice: 0.9066666666666666
Accuracy - window_warp: 0.8522222222222222


100%|██████████| 30/30 [00:00<00:00, 104.03it/s]


Accuracy - spawner: 0.9144444444444444


...

Accuracy - wdba: 0.7444444444444445


100%|██████████| 30/30 [00:00<00:00, 108.56it/s]


Accuracy - random_guided_warp: 0.8844444444444445


100%|██████████| 30/30 [00:03<00:00,  8.41it/s]


Accuracy - discriminative_guided_warp: 0.9177777777777778


# **Các mô hình Deep Learning cơ bản**

### **Thiết kế mô hình Học sâu**

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, Input
from tensorflow.keras.layers import MaxPooling1D, Conv1D
from tensorflow.keras.layers import LSTM, Bidirectional
from tensorflow.keras.layers import BatchNormalization, GlobalAveragePooling1D, Permute, concatenate, Activation, add
import numpy as np
import math

def get_model(model_name, input_shape, nb_class):
    if model_name == "vgg":
        model = cnn_vgg(input_shape, nb_class)
    elif model_name == "lstm1":
        model = lstm1(input_shape, nb_class)
    elif model_name == "lstm":
        model = lstm1v0(input_shape, nb_class)
    elif model_name == "lstm2":
        model = lstm2(input_shape, nb_class)
    elif model_name == "blstm1":
        model = blstm1(input_shape, nb_class)
    elif model_name == "blstm2":
        model = blstm2(input_shape, nb_class)
    elif model_name == "lstmfcn":
        model = lstm_fcn(input_shape, nb_class)
    elif model_name == "resnet":
        model = cnn_resnet(input_shape, nb_class)
    elif model_name == "mlp":
        model = mlp4(input_shape, nb_class)
    elif model_name == "lenet":
        model = cnn_lenet(input_shape, nb_class)
    else:
        print("model name missing")
    return model


def mlp4(input_shape, nb_class):
    # Z. Wang, W. Yan, T. Oates, "Time Series Classification from Scratch with Deep Neural Networks: A Strong Baseline," Int. Joint Conf. Neural Networks, 2017, pp. 1578-1585
    
    ip = Input(shape=input_shape)
    fc = Flatten()(ip)
    
    fc = Dropout(0.1)(fc)
            
    fc = Dense(500, activation='relu')(fc)
    fc = Dropout(0.2)(fc)
    
    fc = Dense(500, activation='relu')(fc)
    fc = Dropout(0.2)(fc)
    
    fc = Dense(500, activation='relu')(fc)
    fc = Dropout(0.3)(fc)
    
    out = Dense(nb_class, activation='softmax')(fc)
    
    model = Model([ip], [out])
    model.summary()
    return model

def cnn_lenet(input_shape, nb_class):
    # Y. Lecun, L. Bottou, Y. Bengio, and P. Haffner, “Gradient-based learning applied to document recognition,” Proceedings of the IEEE, vol. 86, no. 11, pp. 2278–2324, 1998.
    
    ip = Input(shape=input_shape)
    
    conv = ip
    
    nb_cnn = int(round(math.log(input_shape[0], 2))-3)
    print("pooling layers: %d"%nb_cnn)
    
    for i in range(nb_cnn):
        conv = Conv1D(6+10*i, 3, padding='same', activation="relu", kernel_initializer='he_uniform')(conv)
        conv = MaxPooling1D(pool_size=2)(conv)
        
    flat = Flatten()(conv)
    
    fc = Dense(120, activation='relu')(flat)
    fc = Dropout(0.5)(fc)
    
    fc = Dense(84, activation='relu')(fc)
    fc = Dropout(0.5)(fc)
    
    out = Dense(nb_class, activation='softmax')(fc)
    
    model = Model([ip], [out])
    model.summary()
    return model


def cnn_vgg(input_shape, nb_class):
    # K. Simonyan and A. Zisserman, "Very deep convolutional networks for large-scale image recognition," arXiv preprint arXiv:1409.1556, 2014.
    
    ip = Input(shape=input_shape)
    
    conv = ip
    
    nb_cnn = int(round(math.log(input_shape[0], 2))-3)
    print("pooling layers: %d"%nb_cnn)
    
    for i in range(nb_cnn):
        num_filters = min(64*2**i, 512)
        conv = Conv1D(num_filters, 3, padding='same', activation="relu", kernel_initializer='he_uniform')(conv)
        conv = Conv1D(num_filters, 3, padding='same', activation="relu", kernel_initializer='he_uniform')(conv)
        if i > 1:
            conv = Conv1D(num_filters, 3, padding='same', activation="relu", kernel_initializer='he_uniform')(conv)
        conv = MaxPooling1D(pool_size=2)(conv)
        
    flat = Flatten()(conv)
    
    fc = Dense(4096, activation='relu')(flat)
    fc = Dropout(0.5)(fc)
    
    fc = Dense(4096, activation='relu')(fc)
    fc = Dropout(0.5)(fc)
    
    out = Dense(nb_class, activation='softmax')(fc)
    
    model = Model([ip], [out])
    model.summary()
    return model

def lstm1v0(input_shape, nb_class):
    # Original proposal:
    # S. Hochreiter and J. Schmidhuber, “Long Short-Term Memory,” Neural Computation, vol. 9, no. 8, pp. 1735–1780, Nov. 1997.
        
    ip = Input(shape=input_shape)

    l2 = LSTM(512)(ip)
    out = Dense(nb_class, activation='softmax')(l2)

    model = Model([ip], [out])

    model.summary()

    return model

def lstm1(input_shape, nb_class):
    # Original proposal:
    # S. Hochreiter and J. Schmidhuber, “Long Short-Term Memory,” Neural Computation, vol. 9, no. 8, pp. 1735–1780, Nov. 1997.
    
    # Hyperparameter choices: 
    # N. Reimers and I. Gurevych, "Optimal hyperparameters for deep lstm-networks for sequence labeling tasks," arXiv, preprint arXiv:1707.06799, 2017
    
    ip = Input(shape=input_shape)

    l2 = LSTM(100)(ip)
    out = Dense(nb_class, activation='softmax')(l2)

    model = Model([ip], [out])

    model.summary()

    return model


def lstm2(input_shape, nb_class):
    ip = Input(shape=input_shape)

    l1 = LSTM(100, return_sequences=True)(ip)
    l2 = LSTM(100)(l1)
    out = Dense(nb_class, activation='softmax')(l2)

    model = Model([ip], [out])

    model.summary()

    return model


def blstm1(input_shape, nb_class):
    # Original proposal:
    # M. Schuster and K. K. Paliwal, “Bidirectional recurrent neural networks,” IEEE Transactions on Signal Processing, vol. 45, no. 11, pp. 2673–2681, 1997.
    
    # Hyperparameter choices: 
    # N. Reimers and I. Gurevych, "Optimal hyperparameters for deep lstm-networks for sequence labeling tasks," arXiv, preprint arXiv:1707.06799, 2017
    ip = Input(shape=input_shape)

    l2 = Bidirectional(LSTM(100))(ip)
    out = Dense(nb_class, activation='softmax')(l2)

    model = Model([ip], [out])

    model.summary()

    return model

def blstm2(input_shape, nb_class):
    ip = Input(shape=input_shape)

    l1 = Bidirectional(LSTM(100, return_sequences=True))(ip)
    l2 = Bidirectional(LSTM(100))(l1)
    out = Dense(nb_class, activation='softmax')(l2)

    model = Model([ip], [out])

    model.summary()

    return model

def lstm_fcn(input_shape, nb_class):
    # F. Karim, S. Majumdar, H. Darabi, and S. Chen, “LSTM Fully Convolutional Networks for Time Series Classification,” IEEE Access, vol. 6, pp. 1662–1669, 2018.

    ip = Input(shape=input_shape)
    
    # lstm part is a 1 time step multivariate as described in Karim et al. Seems strange, but works I guess.
    lstm = Permute((2, 1))(ip)

    lstm = LSTM(128)(lstm)
    lstm = Dropout(0.8)(lstm)

    conv = Conv1D(128, 8, padding='same', kernel_initializer='he_uniform')(ip)
    conv = BatchNormalization()(conv)
    conv = Activation('relu')(conv)

    conv = Conv1D(256, 5, padding='same', kernel_initializer='he_uniform')(conv)
    conv = BatchNormalization()(conv)
    conv = Activation('relu')(conv)

    conv = Conv1D(128, 3, padding='same', kernel_initializer='he_uniform')(conv)
    conv = BatchNormalization()(conv)
    conv = Activation('relu')(conv)

    flat = GlobalAveragePooling1D()(conv)

    flat = concatenate([lstm, flat])

    out = Dense(nb_class, activation='softmax')(flat)

    model = Model([ip], [out])

    model.summary()

    return model


def cnn_resnet(input_shape, nb_class):
    # I. Fawaz, G. Forestier, J. Weber, L. Idoumghar, P-A Muller, "Data augmentation using synthetic data for time series classification with deep residual networks," International Workshop on Advanced Analytics and Learning on Temporal Data ECML/PKDD, 2018

    ip = Input(shape=input_shape)
    residual = ip
    conv = ip
    
    for i, nb_nodes in enumerate([64, 128, 128]):
        conv = Conv1D(nb_nodes, 8, padding='same', kernel_initializer="glorot_uniform")(conv)
        conv = BatchNormalization()(conv)
        conv = Activation('relu')(conv)

        conv = Conv1D(nb_nodes, 5, padding='same', kernel_initializer="glorot_uniform")(conv)
        conv = BatchNormalization()(conv)
        conv = Activation('relu')(conv)

        conv = Conv1D(nb_nodes, 3, padding='same', kernel_initializer="glorot_uniform")(conv)
        conv = BatchNormalization()(conv)
        conv = Activation('relu')(conv)

        if i < 2:
            # expands dimensions according to Fawaz et al.
            residual = Conv1D(nb_nodes, 1, padding='same', kernel_initializer="glorot_uniform")(residual)
        residual = BatchNormalization()(residual)
        conv = add([residual, conv])
        conv = Activation('relu')(conv)
        
        residual = conv
    
    flat = GlobalAveragePooling1D()(conv)

    out = Dense(nb_class, activation='softmax')(flat)

    model = Model([ip], [out])

    model.summary()

    return model

### **Chuẩn hóa dữ liệu đầu vào**

In [ ]:
def nb_dims(dataset):
    if dataset in ["unipen1a", "unipen1b", "unipen1c"]:
        return 2
    return 1

def nb_classes(dataset):
    if dataset == "CBF":
        return 3 #128
        
def class_offset(y, dataset):
    return (y + class_modifier_add(dataset)) * class_modifier_multi(dataset)
    
def class_modifier_add(dataset):
    if dataset == "CBF":
        return -1 #128
    return 0

def class_modifier_multi(dataset):
    if dataset == "ECG200":
        return 0.5 #96
    if dataset == "FordA":
        return 0.5 #500
    if dataset == "FordB":
        return 0.5 # 500
    if dataset == "Lightning2":
        return 0.5 # 637
    if dataset == "Wafer":
        return 0.5 #152
    return 1

In [ ]:
nb_class = 3
nb_dims = 1
input_shape = (128, 1)

In [ ]:
x_test = x_test.reshape((-1, input_shape[0], input_shape[1])) 
x_train = x_train.reshape((-1, input_shape[0], input_shape[1])) 
y_test = to_categorical(class_offset(y_test, "CBF"), nb_class)
y_train = to_categorical(class_offset(y_train, "CBF"), nb_class)
y_train_new = to_categorical(class_offset(y_train_new, "CBF"), nb_class)

## **Hàm áp dụng dữ liệu sinh từ 12 Phương pháp lần lượt vào mô hình HỌC SÂU**

In [ ]:
def method_apply_deep(model, x_train, y_train, x_test, y_test): 

  # Khong ap dung Sinh du lieu
  model.fit(x_train, y_train, batch_size=batch_size, epochs=nb_epochs)
  yloss, accuracy_None = model.evaluate(x_test, y_test, batch_size=batch_size)

  # Ap dung PP jitter
  x_train_jitter = x_train
  for i in range(0, 99):
    tmp1 = augmentation.jitter(x_train)
    x_train_jitter = np.append(x_train_jitter, tmp1, axis = 0)
  model.fit(x_train_jitter.reshape(3000,128), y_train_new, batch_size=batch_size, epochs=nb_epochs)
  yloss, accuracy_jitter = model.evaluate(x_test, y_test, batch_size=batch_size)

# Ap dung PP scaling
  x_train_scaling = x_train
  for i in range(0, 99):
    tmp1 = augmentation.scaling(x_train)
    x_train_scaling = np.append(x_train_scaling, tmp1, axis = 0)
  model.fit(x_train_scaling.reshape(3000,128), y_train_new, batch_size=batch_size, epochs=nb_epochs)
  yloss, accuracy_scaling = model.evaluate(x_test, y_test, batch_size=batch_size)

  # Ap dung PP rotation
  x_train_rotation = x_train
  for i in range(0, 99):
    tmp1 = augmentation.rotation(x_train)
    x_train_rotation = np.append(x_train_rotation, tmp1, axis = 0)
  model.fit(x_train_rotation.reshape(3000,128), y_train_new, batch_size=batch_size, epochs=nb_epochs)
  yloss, accuracy_rotation = model.evaluate(x_test, y_test, batch_size=batch_size)

  #Ap dung PP permutation
  x_train_permutation = x_train
  for i in range(0, 99):
    tmp1 = augmentation.permutation(x_train)
    x_train_permutation = np.append(x_train_permutation, tmp1, axis = 0)
  model.fit(x_train_permutation.reshape(3000,128), y_train_new, batch_size=batch_size, epochs=nb_epochs)
  yloss, accuracy_permutation = model.evaluate(x_test, y_test, batch_size=batch_size)

  # Ap dung PP magnitude_warp
  x_train_magnitude_warp = x_train
  for i in range(0, 99):
    tmp1 = augmentation.magnitude_warp(x_train)
    x_train_magnitude_warp = np.append(x_train_magnitude_warp, tmp1, axis = 0)
  model.fit(x_train_magnitude_warp.reshape(3000,128), y_train_new, batch_size=batch_size, epochs=nb_epochs)
  yloss, accuracy_magnitude_warp = model.evaluate(x_test, y_test, batch_size=batch_size)

  # Ap dung PP time_warp
  x_train_time_warp = x_train
  for i in range(0, 99):
    tmp1 = augmentation.time_warp(x_train)
    x_train_time_warp = np.append(x_train_time_warp, tmp1, axis = 0)
  model.fit(x_train_jitter.reshape(3000,128), y_train_new, batch_size=batch_size, epochs=nb_epochs)
  yloss, accuracy_time_warp = model.evaluate(x_test, y_test, batch_size=batch_size)

  # Ap dung PP window_slice
  x_train_window_slice = x_train
  for i in range(0, 99):
    tmp1 = augmentation.window_slice(x_train)
    x_train_window_slice = np.append(x_train_window_slice, tmp1, axis = 0)
  model.fit(x_train_window_slice.reshape(3000,128), y_train_new, batch_size=batch_size, epochs=nb_epochs)
  yloss, accuracy_window_slice = model.evaluate(x_test, y_test, batch_size=batch_size)

  # Ap dung PP window_warp
  x_train_window_warp = x_train
  for i in range(0, 99):
    tmp1 = augmentation.window_warp(x_train)
    x_train_window_warp = np.append(x_train_window_warp, tmp1, axis = 0)
  model.fit(x_train_window_warp.reshape(3000,128), y_train_new, batch_size=batch_size, epochs=nb_epochs)
  yloss, accuracy_window_warp = model.evaluate(x_test, y_test, batch_size=batch_size)

  # Ap dung PP spawner
  x_train_spawner = x_train
  for i in range(0, 99):
    tmp1 = spawner(x_train, y_train)
    x_train_spawner = np.append(x_train_spawner, tmp1, axis = 0)
  model.fit(x_train_spawner.reshape(3000,128), y_train_new, batch_size=batch_size, epochs=nb_epochs)
  yloss, accuracy_spawner = model.evaluate(x_test, y_test, batch_size=batch_size)

  # Ap dung PP wdba
  x_train_wdba = x_train
  for i in range(0, 99):
    tmp1 = wdba(x_train, y_train)
    x_train_wdba = np.append(x_train_wdba, tmp1, axis = 0)
  model.fit(x_train_wdba.reshape(3000,128), y_train_new, batch_size=batch_size, epochs=nb_epochs)
  yloss, accuracy_wdba = model.evaluate(x_test, y_test, batch_size=batch_size)

  # Ap dung PP random_guided_warp
  x_train_random_guided_warp = x_train
  for i in range(0, 99):
    tmp1 = random_guided_warp(x_train, y_train)
    x_train_random_guided_warp = np.append(x_train_random_guided_warp, tmp1, axis = 0)
  model.fit(x_train_random_guided_warp.reshape(3000,128), y_train_new, batch_size=batch_size, epochs=nb_epochs)
  yloss, accuracy_random_guided_warp = model.evaluate(x_test, y_test, batch_size=batch_size)

  # Ap dung PP discriminative_guided_warp
  x_train_discriminative_guided_warp = x_train
  for i in range(0, 99):
    tmp1 = discriminative_guided_warp(x_train, y_train)
    x_train_discriminative_guided_warp = np.append(x_train_discriminative_guided_warp, tmp1, axis = 0)
  model.fit(x_train_discriminative_guided_warp.reshape(3000,128), y_train_new, batch_size=batch_size, epochs=nb_epochs)
  yloss, accuracy_discriminative_guided_warp = model.evaluate(x_test, y_test, batch_size=batch_size)

  print("Accuracy - None:: ", accuracy_None)
  print("Accuracy - jitter:: ", accuracy_jitter)
  print("Accuracy - scaling:: ", accuracy_scaling)
  print("Accuracy - rotation:: ", accuracy_rotation)
  print("Accuracy - permutation:: ", accuracy_permutation)
  print("Accuracy - magnitude_warp:: ", accuracy_magnitude_warp)
  print("Accuracy - time_warp:: ", accuracy_time_warp)
  print("Accuracy - window_slice:: ", accuracy_window_slice)
  print("Accuracy - window_warp:: ", accuracy_window_warp)
  print("Accuracy - spawner:: ", accuracy_spawner)
  print("Accuracy - wdba:: ", accuracy_wdba)
  print("Accuracy - random_guided_warp:: ", accuracy_random_guided_warp)
  print("Accuracy - discriminative_guided_warp:: ", accuracy_discriminative_guided_warp)


**Thiết lập thông số mô hình**

In [ ]:
nb_iterations = 1000
batch_size = 256
# nb_epochs = np.ceil(nb_iterations * (batch_size / x_train.shape[0])).astype(int)
nb_epochs = 5
lr =1e-2

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='acc', factor=0.1, patience=np.ceil(nb_epochs/20.).astype(int), verbose=0, mode='auto', min_lr=1e-5, cooldown=np.ceil(nb_epochs/40.).astype(int))

In [ ]:
optimizer = "sgd"
if optimizer=="adam":
        from tensorflow.keras.optimizers import Adam
        optm = Adam(lr=lr)
elif optimizer=="nadam":
    from tensorflow.keras.optimizers import Nadam
    optm = Nadam(lr=lr)
elif optimizer=="adadelta":
    from tensorflow.keras.optimizers import Adadelta
    optm = Adadelta(lr=lr, rho=0.95, epsilon=1e-8)
else:
    from tensorflow.keras.optimizers import SGD
    optm = SGD(lr=lr, decay=5e-4, momentum=0.9) #, nesterov=True)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


**vgg**

In [ ]:
model = get_model("vgg", input_shape, nb_class)
model.compile(optimizer=optm, loss='categorical_crossentropy', metrics=['accuracy'])
method_apply_deep(model, x_train, y_train, x_test, y_test)

pooling layers: 4
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 1)]          0         
                                                                 
 conv1d (Conv1D)             (None, 128, 64)           256       
                                                                 
 conv1d_1 (Conv1D)           (None, 128, 64)           12352     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 64, 64)           0         
 )                                                               
                                                                 
 conv1d_2 (Conv1D)           (None, 64, 128)           24704     
                                                                 
 conv1d_3 (Conv1D)           (None, 64, 128)           49280     
                                           

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


Epoch 1/5
12/12 [==============================] - 47s 4s/step - loss: 1.4791 - accuracy: 0.4710
Epoch 2/5
12/12 [==============================] - 47s 4s/step - loss: 0.9551 - accuracy: 0.5193
Epoch 3/5
12/12 [==============================] - 47s 4s/step - loss: 0.7062 - accuracy: 0.7203
Epoch 4/5
12/12 [==============================] - 47s 4s/step - loss: 0.2897 - accuracy: 0.9103
Epoch 5/5
4/4 [==============================] - 3s 607ms/step - loss: 0.1730 - accuracy: 0.9756
Epoch 1/5
12/12 [==============================] - 47s 4s/step - loss: 0.1956 - accuracy: 0.9643
Epoch 2/5
12/12 [==============================] - 47s 4s/step - loss: 0.0212 - accuracy: 0.9923
Epoch 3/5
12/12 [==============================] - 47s 4s/step - loss: 0.0126 - accuracy: 0.9963
Epoch 4/5
12/12 [==============================] - 47s 4s/step - loss: 0.0145 - accuracy: 0.9953
Epoch 5/5
4/4 [==============================] - 3s 607ms/step - loss: 0.0456 - accuracy: 0.9878
Epoch 1/5
12/12 [=============

100%|██████████| 30/30 [00:00<00:00, 96.68it/s]


Epoch 1/5
12/12 [==============================] - 47s 4s/step - loss: 0.8070 - accuracy: 0.7850
Epoch 2/5
12/12 [==============================] - 47s 4s/step - loss: 0.4927 - accuracy: 0.8140
Epoch 3/5
12/12 [==============================] - 47s 4s/step - loss: 0.3298 - accuracy: 0.8533
Epoch 4/5
12/12 [==============================] - 47s 4s/step - loss: 0.2610 - accuracy: 0.8927
Epoch 5/5
4/4 [==============================] - 3s 612ms/step - loss: 0.1409 - accuracy: 0.9589


...

Epoch 1/5
12/12 [==============================] - 47s 4s/step - loss: 0.0117 - accuracy: 0.9997
Epoch 2/5
12/12 [==============================] - 47s 4s/step - loss: 0.0020 - accuracy: 1.0000
Epoch 3/5
12/12 [==============================] - 47s 4s/step - loss: 2.1463e-04 - accuracy: 1.0000
Epoch 4/5
12/12 [==============================] - 47s 4s/step - loss: 2.1800e-04 - accuracy: 1.0000
Epoch 5/5
4/4 [==============================] - 3s 610ms/step - loss: 0.0792 - accuracy: 0.9656


100%|██████████| 30/30 [00:00<00:00, 86.37it/s]


Epoch 1/5
12/12 [==============================] - 47s 4s/step - loss: 0.0340 - accuracy: 0.9843
Epoch 2/5
12/12 [==============================] - 47s 4s/step - loss: 0.0093 - accuracy: 0.9987
Epoch 3/5
12/12 [==============================] - 47s 4s/step - loss: 0.0047 - accuracy: 0.9997
Epoch 4/5
12/12 [==============================] - 47s 4s/step - loss: 0.0026 - accuracy: 1.0000
Epoch 5/5
4/4 [==============================] - 3s 606ms/step - loss: 0.0591 - accuracy: 0.9789


100%|██████████| 30/30 [00:04<00:00,  7.42it/s]


Epoch 1/5
12/12 [==============================] - 47s 4s/step - loss: 0.0019 - accuracy: 0.9997
Epoch 2/5
12/12 [==============================] - 47s 4s/step - loss: 0.0027 - accuracy: 0.9993
Epoch 3/5
12/12 [==============================] - 47s 4s/step - loss: 0.0017 - accuracy: 1.0000
Epoch 4/5
12/12 [==============================] - 47s 4s/step - loss: 8.3627e-04 - accuracy: 0.9997
Epoch 5/5
4/4 [==============================] - 3s 610ms/step - loss: 0.0976 - accuracy: 0.9722
Accuracy - None::  0.41333332657814026
Accuracy - jitter::  0.996666669845581
Accuracy - scaling::  0.995555579662323
Accuracy - rotation::  0.8355555534362793
Accuracy - permutation::  0.9755555391311646
Accuracy - magnitude_warp::  0.9877777695655823
Accuracy - time_warp::  0.9888888597488403
Accuracy - window_slice::  0.9877777695655823
Accuracy - window_warp::  0.9822221994400024
Accuracy - spawner::  0.9588888883590698
Accuracy - wdba::  0.9655555486679077
Accuracy - random_guided_warp::  0.9788888692

**lstm1**

In [ ]:
model = get_model("lstm1", input_shape, nb_class)
model.compile(optimizer=optm, loss='categorical_crossentropy', metrics=['accuracy'])
method_apply_deep(model, x_train, y_train, x_test, y_test)

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 128, 1)]          0         
                                                                 
 lstm (LSTM)                 (None, 100)               40800     
                                                                 
 dense_3 (Dense)             (None, 3)                 303       
                                                                 
Total params: 41,103
Trainable params: 41,103
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
1/1 [==============================] - 2s 2s/step - loss: 1.0986 - accuracy: 0.4000
Epoch 2/5
1/1 [==============================] - 0s 95ms/step - loss: 1.0984 - accuracy: 0.4000
Epoch 3/5
1/1 [==============================] - 0s 91ms/step - loss: 1.0980 - accuracy: 0.4000
Epoch 4/5
1/1 [==================

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


Epoch 1/5
12/12 [==============================] - 5s 385ms/step - loss: 1.0850 - accuracy: 0.4000
Epoch 2/5
12/12 [==============================] - 5s 387ms/step - loss: 1.0845 - accuracy: 0.4000
Epoch 3/5
12/12 [==============================] - 5s 385ms/step - loss: 1.0843 - accuracy: 0.4000
Epoch 4/5
12/12 [==============================] - 5s 384ms/step - loss: 1.0842 - accuracy: 0.4000
Epoch 5/5
4/4 [==============================] - 1s 132ms/step - loss: 1.1023 - accuracy: 0.3311
Epoch 1/5
12/12 [==============================] - 5s 392ms/step - loss: 1.0727 - accuracy: 0.4000
Epoch 2/5
12/12 [==============================] - 5s 384ms/step - loss: 1.0679 - accuracy: 0.4320
Epoch 3/5
12/12 [==============================] - 5s 390ms/step - loss: 1.0582 - accuracy: 0.4997
Epoch 4/5
12/12 [==============================] - 5s 380ms/step - loss: 1.0412 - accuracy: 0.5200
Epoch 5/5
4/4 [==============================] - 1s 134ms/step - loss: 0.9885 - accuracy: 0.4433
Epoch 1/5
12/1

100%|██████████| 30/30 [00:00<00:00, 98.70it/s] 


Epoch 1/5
12/12 [==============================] - 5s 390ms/step - loss: 1.0721 - accuracy: 0.4123
Epoch 2/5
12/12 [==============================] - 5s 384ms/step - loss: 1.0420 - accuracy: 0.5163
Epoch 3/5
12/12 [==============================] - 5s 382ms/step - loss: 1.0353 - accuracy: 0.5053
Epoch 4/5
12/12 [==============================] - 5s 387ms/step - loss: 1.0268 - accuracy: 0.5607
Epoch 5/5
4/4 [==============================] - 1s 142ms/step - loss: 1.0461 - accuracy: 0.4678


...

Epoch 1/5
12/12 [==============================] - 5s 387ms/step - loss: 1.0170 - accuracy: 0.6400
Epoch 2/5
12/12 [==============================] - 5s 387ms/step - loss: 0.9967 - accuracy: 0.6743
Epoch 3/5
12/12 [==============================] - 5s 385ms/step - loss: 0.9655 - accuracy: 0.7137
Epoch 4/5
12/12 [==============================] - 5s 381ms/step - loss: 0.9043 - accuracy: 0.6660
Epoch 5/5
4/4 [==============================] - 1s 135ms/step - loss: 1.0575 - accuracy: 0.4778


100%|██████████| 30/30 [00:00<00:00, 100.36it/s]


Epoch 1/5
12/12 [==============================] - 5s 385ms/step - loss: 0.9694 - accuracy: 0.6227
Epoch 2/5
12/12 [==============================] - 5s 383ms/step - loss: 0.9362 - accuracy: 0.6370
Epoch 3/5
12/12 [==============================] - 5s 386ms/step - loss: 0.9038 - accuracy: 0.6350
Epoch 4/5
12/12 [==============================] - 5s 384ms/step - loss: 0.8820 - accuracy: 0.6730
Epoch 5/5
4/4 [==============================] - 1s 142ms/step - loss: 0.9053 - accuracy: 0.5489


100%|██████████| 30/30 [00:03<00:00,  7.63it/s]


Epoch 1/5
12/12 [==============================] - 5s 386ms/step - loss: 0.8277 - accuracy: 0.6927
Epoch 2/5
12/12 [==============================] - 5s 383ms/step - loss: 0.8119 - accuracy: 0.6800
Epoch 3/5
12/12 [==============================] - 5s 385ms/step - loss: 0.8175 - accuracy: 0.6720
Epoch 4/5
12/12 [==============================] - 5s 382ms/step - loss: 0.7328 - accuracy: 0.7277
Epoch 5/5
4/4 [==============================] - 1s 133ms/step - loss: 1.0023 - accuracy: 0.5300
Accuracy - None::  0.3311111032962799
Accuracy - jitter::  0.42888888716697693
Accuracy - scaling::  0.5333333611488342
Accuracy - rotation::  0.3311111032962799
Accuracy - permutation::  0.3311111032962799
Accuracy - magnitude_warp::  0.44333332777023315
Accuracy - time_warp::  0.6100000143051147
Accuracy - window_slice::  0.6855555772781372
Accuracy - window_warp::  0.4699999988079071
Accuracy - spawner::  0.4677777886390686
Accuracy - wdba::  0.47777777910232544
Accuracy - random_guided_warp::  0.54

**lstm**

In [ ]:
model = get_model("lstm", input_shape, nb_class)
model.compile(optimizer=optm, loss='categorical_crossentropy', metrics=['accuracy'])
method_apply_deep(model, x_train, y_train, x_test, y_test)

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 128, 1)]          0         
                                                                 
 lstm_1 (LSTM)               (None, 512)               1052672   
                                                                 
 dense_4 (Dense)             (None, 3)                 1539      
                                                                 
Total params: 1,054,211
Trainable params: 1,054,211
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
1/1 [==============================] - 3s 3s/step - loss: 1.0970 - accuracy: 0.3667
Epoch 2/5
1/1 [==============================] - 1s 1s/step - loss: 1.0968 - accuracy: 0.3333
Epoch 3/5
1/1 [==============================] - 1s 1s/step - loss: 1.0964 - accuracy: 0.3333
Epoch 4/5
1/1 [================

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


Epoch 1/5
12/12 [==============================] - 55s 5s/step - loss: 1.0845 - accuracy: 0.4467
Epoch 2/5
12/12 [==============================] - 55s 5s/step - loss: 1.0839 - accuracy: 0.4000
Epoch 3/5
12/12 [==============================] - 55s 5s/step - loss: 1.0838 - accuracy: 0.4000
Epoch 4/5
12/12 [==============================] - 54s 5s/step - loss: 1.0837 - accuracy: 0.4000
Epoch 5/5
4/4 [==============================] - 5s 1s/step - loss: 1.0982 - accuracy: 0.3311
Epoch 1/5
12/12 [==============================] - 54s 5s/step - loss: 1.0718 - accuracy: 0.4020
Epoch 2/5
12/12 [==============================] - 54s 5s/step - loss: 1.0688 - accuracy: 0.4200
Epoch 3/5
12/12 [==============================] - 54s 5s/step - loss: 1.0635 - accuracy: 0.4737
Epoch 4/5
12/12 [==============================] - 54s 5s/step - loss: 1.0551 - accuracy: 0.5080
Epoch 5/5
4/4 [==============================] - 5s 1s/step - loss: 1.0490 - accuracy: 0.5467
Epoch 1/5
12/12 [===================

100%|██████████| 30/30 [00:00<00:00, 81.44it/s]


Epoch 1/5
12/12 [==============================] - 54s 5s/step - loss: 0.6356 - accuracy: 0.7177
Epoch 2/5
12/12 [==============================] - 54s 5s/step - loss: 0.6289 - accuracy: 0.7267
Epoch 3/5
12/12 [==============================] - 54s 4s/step - loss: 0.6207 - accuracy: 0.7260
Epoch 4/5
12/12 [==============================] - 54s 4s/step - loss: 0.6126 - accuracy: 0.7327
Epoch 5/5
4/4 [==============================] - 5s 1s/step - loss: 0.4569 - accuracy: 0.8167


...

Epoch 1/5
12/12 [==============================] - 53s 4s/step - loss: 0.3445 - accuracy: 0.9287
Epoch 2/5
12/12 [==============================] - 53s 4s/step - loss: 1.4270 - accuracy: 0.4257
Epoch 3/5
12/12 [==============================] - 53s 4s/step - loss: 1.0905 - accuracy: 0.3583
Epoch 4/5
12/12 [==============================] - 53s 4s/step - loss: 1.0690 - accuracy: 0.4740
Epoch 5/5
4/4 [==============================] - 5s 1s/step - loss: 1.0882 - accuracy: 0.3633


100%|██████████| 30/30 [00:00<00:00, 90.94it/s]


Epoch 1/5
12/12 [==============================] - 54s 4s/step - loss: 1.0696 - accuracy: 0.4120
Epoch 2/5
12/12 [==============================] - 54s 5s/step - loss: 1.0678 - accuracy: 0.4263
Epoch 3/5
12/12 [==============================] - 54s 5s/step - loss: 1.0665 - accuracy: 0.4837
Epoch 4/5
12/12 [==============================] - 55s 5s/step - loss: 1.0654 - accuracy: 0.5330
Epoch 5/5
4/4 [==============================] - 5s 1s/step - loss: 1.0817 - accuracy: 0.4844


100%|██████████| 30/30 [00:03<00:00,  7.62it/s]


Epoch 1/5
12/12 [==============================] - 53s 4s/step - loss: 1.0596 - accuracy: 0.5270
Epoch 2/5
12/12 [==============================] - 53s 4s/step - loss: 1.0571 - accuracy: 0.5520
Epoch 3/5
12/12 [==============================] - 53s 4s/step - loss: 1.0538 - accuracy: 0.5743
Epoch 4/5
12/12 [==============================] - 53s 4s/step - loss: 1.0498 - accuracy: 0.5900
Epoch 5/5
4/4 [==============================] - 5s 1s/step - loss: 1.0629 - accuracy: 0.5089
Accuracy - None::  0.5111111402511597
Accuracy - jitter::  0.33444443345069885
Accuracy - scaling::  0.5266666412353516
Accuracy - rotation::  0.5199999809265137
Accuracy - permutation::  0.3311111032962799
Accuracy - magnitude_warp::  0.54666668176651
Accuracy - time_warp::  0.5722222328186035
Accuracy - window_slice::  0.5888888835906982
Accuracy - window_warp::  0.7844444513320923
Accuracy - spawner::  0.8166666626930237
Accuracy - wdba::  0.3633333444595337
Accuracy - random_guided_warp::  0.48444443941116333

**lstm2**

In [ ]:
model = get_model("lstm2", input_shape, nb_class)
model.compile(optimizer=optm, loss='categorical_crossentropy', metrics=['accuracy'])
method_apply_deep(model, x_train, y_train, x_test, y_test)

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 128, 1)]          0         
                                                                 
 lstm_2 (LSTM)               (None, 128, 100)          40800     
                                                                 
 lstm_3 (LSTM)               (None, 100)               80400     
                                                                 
 dense_5 (Dense)             (None, 3)                 303       
                                                                 
Total params: 121,503
Trainable params: 121,503
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
1/1 [==============================] - 4s 4s/step - loss: 1.0995 - accuracy: 0.3333
Epoch 2/5
1/1 [==============================] - 0s 205ms/step - loss: 1.0993 - accuracy: 0

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


Epoch 1/5
12/12 [==============================] - 11s 929ms/step - loss: 1.0820 - accuracy: 0.4933
Epoch 2/5
12/12 [==============================] - 11s 942ms/step - loss: 1.0816 - accuracy: 0.4417
Epoch 3/5
12/12 [==============================] - 11s 931ms/step - loss: 1.0815 - accuracy: 0.4143
Epoch 4/5
12/12 [==============================] - 11s 932ms/step - loss: 1.0816 - accuracy: 0.4030
Epoch 5/5
4/4 [==============================] - 1s 305ms/step - loss: 1.0902 - accuracy: 0.4444
Epoch 1/5
12/12 [==============================] - 11s 929ms/step - loss: 1.0647 - accuracy: 0.4913
Epoch 2/5
12/12 [==============================] - 11s 937ms/step - loss: 1.0594 - accuracy: 0.5043
Epoch 3/5
12/12 [==============================] - 11s 941ms/step - loss: 1.0506 - accuracy: 0.5240
Epoch 4/5
12/12 [==============================] - 11s 941ms/step - loss: 1.0377 - accuracy: 0.5140
Epoch 5/5
4/4 [==============================] - 1s 305ms/step - loss: 1.0267 - accuracy: 0.4033
Epoch 

100%|██████████| 30/30 [00:00<00:00, 89.52it/s]


Epoch 1/5
12/12 [==============================] - 11s 943ms/step - loss: 0.6885 - accuracy: 0.6890
Epoch 2/5
12/12 [==============================] - 11s 940ms/step - loss: 0.6505 - accuracy: 0.6837
Epoch 3/5
12/12 [==============================] - 11s 940ms/step - loss: 0.6513 - accuracy: 0.6840
Epoch 4/5
12/12 [==============================] - 11s 942ms/step - loss: 0.6266 - accuracy: 0.6917
Epoch 5/5
4/4 [==============================] - 1s 299ms/step - loss: 0.5275 - accuracy: 0.7467


...

Epoch 1/5
12/12 [==============================] - 11s 936ms/step - loss: 0.2862 - accuracy: 0.9673
Epoch 2/5
12/12 [==============================] - 11s 937ms/step - loss: 0.2045 - accuracy: 0.9673
Epoch 3/5
12/12 [==============================] - 11s 937ms/step - loss: 0.1487 - accuracy: 0.9650
Epoch 4/5
12/12 [==============================] - 11s 931ms/step - loss: 0.1253 - accuracy: 0.9653
Epoch 5/5
4/4 [==============================] - 1s 300ms/step - loss: 0.4808 - accuracy: 0.8244


100%|██████████| 30/30 [00:00<00:00, 95.85it/s]


Epoch 1/5
12/12 [==============================] - 11s 935ms/step - loss: 1.4236 - accuracy: 0.5620
Epoch 2/5
12/12 [==============================] - 11s 931ms/step - loss: 0.8372 - accuracy: 0.6727
Epoch 3/5
12/12 [==============================] - 11s 936ms/step - loss: 0.6405 - accuracy: 0.6793
Epoch 4/5
12/12 [==============================] - 11s 932ms/step - loss: 0.5388 - accuracy: 0.7230
Epoch 5/5
4/4 [==============================] - 1s 298ms/step - loss: 0.4882 - accuracy: 0.7578


100%|██████████| 30/30 [00:04<00:00,  7.50it/s]


Epoch 1/5
12/12 [==============================] - 11s 938ms/step - loss: 0.4051 - accuracy: 0.8137
Epoch 2/5
12/12 [==============================] - 11s 932ms/step - loss: 0.4247 - accuracy: 0.8130
Epoch 3/5
12/12 [==============================] - 11s 932ms/step - loss: 0.3782 - accuracy: 0.8313
Epoch 4/5
12/12 [==============================] - 11s 941ms/step - loss: 0.3573 - accuracy: 0.8473
Epoch 5/5
4/4 [==============================] - 1s 300ms/step - loss: 0.3492 - accuracy: 0.8744
Accuracy - None::  0.3311111032962799
Accuracy - jitter::  0.3311111032962799
Accuracy - scaling::  0.5188888907432556
Accuracy - rotation::  0.5144444704055786
Accuracy - permutation::  0.4444444477558136
Accuracy - magnitude_warp::  0.4033333361148834
Accuracy - time_warp::  0.551111102104187
Accuracy - window_slice::  0.6166666746139526
Accuracy - window_warp::  0.6744444370269775
Accuracy - spawner::  0.746666669845581
Accuracy - wdba::  0.8244444727897644
Accuracy - random_guided_warp::  0.757

**blstm1**

In [ ]:
model = get_model("blstm1", input_shape, nb_class)
model.compile(optimizer=optm, loss='categorical_crossentropy', metrics=['accuracy'])
method_apply_deep(model, x_train, y_train, x_test, y_test)

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 128, 1)]          0         
                                                                 
 bidirectional (Bidirectiona  (None, 200)              81600     
 l)                                                              
                                                                 
 dense_6 (Dense)             (None, 3)                 603       
                                                                 
Total params: 82,203
Trainable params: 82,203
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
1/1 [==============================] - 4s 4s/step - loss: 1.1013 - accuracy: 0.4000
Epoch 2/5
1/1 [==============================] - 0s 141ms/step - loss: 1.1012 - accuracy: 0.4000
Epoch 3/5
1/1 [==============================] - 0s 138ms/step

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


Epoch 1/5
12/12 [==============================] - 9s 741ms/step - loss: 1.0854 - accuracy: 0.4823
Epoch 2/5
12/12 [==============================] - 9s 731ms/step - loss: 1.0846 - accuracy: 0.4547
Epoch 3/5
12/12 [==============================] - 9s 724ms/step - loss: 1.0841 - accuracy: 0.4000
Epoch 4/5
12/12 [==============================] - 9s 728ms/step - loss: 1.0839 - accuracy: 0.4000
Epoch 5/5
4/4 [==============================] - 1s 213ms/step - loss: 1.0983 - accuracy: 0.3367
Epoch 1/5
12/12 [==============================] - 9s 736ms/step - loss: 1.0695 - accuracy: 0.4080
Epoch 2/5
12/12 [==============================] - 9s 733ms/step - loss: 1.0676 - accuracy: 0.4090
Epoch 3/5
12/12 [==============================] - 9s 734ms/step - loss: 1.0651 - accuracy: 0.4107
Epoch 4/5
12/12 [==============================] - 9s 728ms/step - loss: 1.0622 - accuracy: 0.4293
Epoch 5/5
4/4 [==============================] - 1s 215ms/step - loss: 1.0851 - accuracy: 0.4411
Epoch 1/5
12/1

100%|██████████| 30/30 [00:00<00:00, 92.81it/s]


Epoch 1/5
12/12 [==============================] - 9s 728ms/step - loss: 0.8221 - accuracy: 0.7347
Epoch 2/5
12/12 [==============================] - 9s 738ms/step - loss: 0.7820 - accuracy: 0.6953
Epoch 3/5
12/12 [==============================] - 9s 728ms/step - loss: 0.7458 - accuracy: 0.6940
Epoch 4/5
12/12 [==============================] - 9s 723ms/step - loss: 0.7056 - accuracy: 0.6973
Epoch 5/5
4/4 [==============================] - 1s 219ms/step - loss: 0.6191 - accuracy: 0.6989


...

Epoch 1/5
12/12 [==============================] - 9s 747ms/step - loss: 0.3974 - accuracy: 0.9490
Epoch 2/5
12/12 [==============================] - 9s 721ms/step - loss: 0.3322 - accuracy: 0.9483
Epoch 3/5
12/12 [==============================] - 9s 731ms/step - loss: 0.2739 - accuracy: 0.9523
Epoch 4/5
12/12 [==============================] - 9s 746ms/step - loss: 0.2426 - accuracy: 0.9503
Epoch 5/5
4/4 [==============================] - 1s 216ms/step - loss: 0.5019 - accuracy: 0.7667


100%|██████████| 30/30 [00:00<00:00, 77.92it/s]


Epoch 1/5
12/12 [==============================] - 9s 775ms/step - loss: 1.0301 - accuracy: 0.6353
Epoch 2/5
12/12 [==============================] - 9s 778ms/step - loss: 0.9278 - accuracy: 0.6110
Epoch 3/5
12/12 [==============================] - 9s 752ms/step - loss: 0.6027 - accuracy: 0.7380
Epoch 4/5
12/12 [==============================] - 9s 746ms/step - loss: 0.5543 - accuracy: 0.7760
Epoch 5/5
4/4 [==============================] - 1s 214ms/step - loss: 0.5495 - accuracy: 0.7456


100%|██████████| 30/30 [00:04<00:00,  7.46it/s]


Epoch 1/5
12/12 [==============================] - 9s 732ms/step - loss: 0.4379 - accuracy: 0.8427
Epoch 2/5
12/12 [==============================] - 9s 745ms/step - loss: 0.4153 - accuracy: 0.8527
Epoch 3/5
12/12 [==============================] - 9s 732ms/step - loss: 0.3891 - accuracy: 0.8547
Epoch 4/5
12/12 [==============================] - 9s 731ms/step - loss: 0.3640 - accuracy: 0.8660
Epoch 5/5
4/4 [==============================] - 1s 217ms/step - loss: 0.4144 - accuracy: 0.8144
Accuracy - None::  0.32777777314186096
Accuracy - jitter::  0.3311111032962799
Accuracy - scaling::  0.35111111402511597
Accuracy - rotation::  0.5166666507720947
Accuracy - permutation::  0.33666667342185974
Accuracy - magnitude_warp::  0.44111111760139465
Accuracy - time_warp::  0.5144444704055786
Accuracy - window_slice::  0.4866666793823242
Accuracy - window_warp::  0.7155555486679077
Accuracy - spawner::  0.698888897895813
Accuracy - wdba::  0.7666666507720947
Accuracy - random_guided_warp::  0.74

**blstm2**

In [ ]:
model = get_model("blstm2", input_shape, nb_class)
model.compile(optimizer=optm, loss='categorical_crossentropy', metrics=['accuracy'])
method_apply_deep(model, x_train, y_train, x_test, y_test)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 1)]          0         
                                                                 
 bidirectional (Bidirectiona  (None, 128, 200)         81600     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 200)              240800    
 nal)                                                            
                                                                 
 dense (Dense)               (None, 3)                 603       
                                                                 
Total params: 323,003
Trainable params: 323,003
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
1/1 [==============================] - 8s 8s/

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


Epoch 1/5
12/12 [==============================] - 27s 2s/step - loss: 1.0085 - accuracy: 0.5197
Epoch 2/5
12/12 [==============================] - 28s 2s/step - loss: 0.8956 - accuracy: 0.5723
Epoch 3/5
12/12 [==============================] - 27s 2s/step - loss: 0.8591 - accuracy: 0.5983
Epoch 4/5
12/12 [==============================] - 26s 2s/step - loss: 0.8325 - accuracy: 0.6280
Epoch 5/5
4/4 [==============================] - 3s 599ms/step - loss: 0.5598 - accuracy: 0.7689
Epoch 1/5
12/12 [==============================] - 28s 2s/step - loss: 0.6617 - accuracy: 0.7163
Epoch 2/5
12/12 [==============================] - 26s 2s/step - loss: 0.4567 - accuracy: 0.8020
Epoch 3/5
12/12 [==============================] - 26s 2s/step - loss: 0.3570 - accuracy: 0.8477
Epoch 4/5
12/12 [==============================] - 26s 2s/step - loss: 0.2736 - accuracy: 0.9050
Epoch 5/5
4/4 [==============================] - 2s 595ms/step - loss: 0.1838 - accuracy: 0.9433
Epoch 1/5
12/12 [=============

100%|██████████| 30/30 [00:00<00:00, 95.81it/s]


Epoch 1/5
12/12 [==============================] - 27s 2s/step - loss: 0.8200 - accuracy: 0.7783
Epoch 2/5
12/12 [==============================] - 26s 2s/step - loss: 0.5634 - accuracy: 0.8160
Epoch 3/5
12/12 [==============================] - 25s 2s/step - loss: 0.4776 - accuracy: 0.8180
Epoch 4/5
12/12 [==============================] - 27s 2s/step - loss: 0.4348 - accuracy: 0.8257
Epoch 5/5
4/4 [==============================] - 2s 591ms/step - loss: 0.1438 - accuracy: 0.9878


...

Epoch 1/5
12/12 [==============================] - 26s 2s/step - loss: 0.0606 - accuracy: 0.9997
Epoch 2/5
12/12 [==============================] - 25s 2s/step - loss: 0.0255 - accuracy: 0.9997
Epoch 3/5
12/12 [==============================] - 25s 2s/step - loss: 0.0166 - accuracy: 0.9997
Epoch 4/5
12/12 [==============================] - 25s 2s/step - loss: 0.0125 - accuracy: 0.9997
Epoch 5/5
4/4 [==============================] - 2s 580ms/step - loss: 0.0995 - accuracy: 0.9789


100%|██████████| 30/30 [00:00<00:00, 94.07it/s]


Epoch 1/5
12/12 [==============================] - 26s 2s/step - loss: 0.1992 - accuracy: 0.9353
Epoch 2/5
12/12 [==============================] - 26s 2s/step - loss: 0.1398 - accuracy: 0.9530
Epoch 3/5
12/12 [==============================] - 26s 2s/step - loss: 0.0927 - accuracy: 0.9663
Epoch 4/5
12/12 [==============================] - 26s 2s/step - loss: 0.0810 - accuracy: 0.9740
Epoch 5/5
4/4 [==============================] - 2s 582ms/step - loss: 0.0907 - accuracy: 0.9678


100%|██████████| 30/30 [00:03<00:00,  7.97it/s]


Epoch 1/5
12/12 [==============================] - 27s 2s/step - loss: 0.0413 - accuracy: 0.9930
Epoch 2/5
12/12 [==============================] - 26s 2s/step - loss: 0.0356 - accuracy: 0.9920
Epoch 3/5
12/12 [==============================] - 27s 2s/step - loss: 0.0326 - accuracy: 0.9940
Epoch 4/5
12/12 [==============================] - 27s 2s/step - loss: 0.0293 - accuracy: 0.9933
Epoch 5/5
4/4 [==============================] - 3s 601ms/step - loss: 0.1057 - accuracy: 0.9578
Accuracy - None::  0.4211111068725586
Accuracy - jitter::  0.47999998927116394
Accuracy - scaling::  0.7622222304344177
Accuracy - rotation::  0.8122222423553467
Accuracy - permutation::  0.7688888907432556
Accuracy - magnitude_warp::  0.9433333277702332
Accuracy - time_warp::  0.9688888788223267
Accuracy - window_slice::  0.9322222471237183
Accuracy - window_warp::  0.9688888788223267
Accuracy - spawner::  0.9877777695655823
Accuracy - wdba::  0.9788888692855835
Accuracy - random_guided_warp::  0.967777788639

**lstmfcn**

In [ ]:
model = get_model("lstmfcn", input_shape, nb_class)
model.compile(optimizer=optm, loss='categorical_crossentropy', metrics=['accuracy'])
method_apply_deep(model, x_train, y_train, x_test, y_test)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 128, 1)]     0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 128, 128)     1152        ['input_2[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 128, 128)    512         ['conv1d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 128, 128)     0           ['batch_normalization[0][0]

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


Epoch 1/5
12/12 [==============================] - 16s 1s/step - loss: 0.4482 - accuracy: 0.8743
Epoch 2/5
12/12 [==============================] - 17s 1s/step - loss: 0.2606 - accuracy: 0.9637
Epoch 3/5
12/12 [==============================] - 16s 1s/step - loss: 0.1733 - accuracy: 0.9787
Epoch 4/5
12/12 [==============================] - 16s 1s/step - loss: 0.1242 - accuracy: 0.9920
Epoch 5/5
4/4 [==============================] - 1s 272ms/step - loss: 0.6206 - accuracy: 0.7033
Epoch 1/5
12/12 [==============================] - 16s 1s/step - loss: 0.1169 - accuracy: 0.9817
Epoch 2/5
12/12 [==============================] - 16s 1s/step - loss: 0.0988 - accuracy: 0.9857
Epoch 3/5
12/12 [==============================] - 16s 1s/step - loss: 0.0814 - accuracy: 0.9900
Epoch 4/5
12/12 [==============================] - 16s 1s/step - loss: 0.0712 - accuracy: 0.9910
Epoch 5/5
4/4 [==============================] - 1s 273ms/step - loss: 0.9240 - accuracy: 0.5689
Epoch 1/5
12/12 [=============

100%|██████████| 30/30 [00:00<00:00, 99.19it/s]


Epoch 1/5
12/12 [==============================] - 16s 1s/step - loss: 0.4128 - accuracy: 0.8190
Epoch 2/5
12/12 [==============================] - 16s 1s/step - loss: 0.3322 - accuracy: 0.8450
Epoch 3/5
12/12 [==============================] - 16s 1s/step - loss: 0.3110 - accuracy: 0.8590
Epoch 4/5
12/12 [==============================] - 17s 1s/step - loss: 0.2984 - accuracy: 0.8670
Epoch 5/5
4/4 [==============================] - 1s 273ms/step - loss: 0.8330 - accuracy: 0.5978


...

Epoch 1/5
12/12 [==============================] - 17s 1s/step - loss: 0.0153 - accuracy: 1.0000
Epoch 2/5
12/12 [==============================] - 17s 1s/step - loss: 0.0121 - accuracy: 0.9997
Epoch 3/5
12/12 [==============================] - 17s 1s/step - loss: 0.0106 - accuracy: 0.9997
Epoch 4/5
12/12 [==============================] - 17s 1s/step - loss: 0.0083 - accuracy: 0.9997
Epoch 5/5
4/4 [==============================] - 1s 273ms/step - loss: 0.0953 - accuracy: 0.9911


100%|██████████| 30/30 [00:00<00:00, 103.78it/s]


Epoch 1/5
12/12 [==============================] - 16s 1s/step - loss: 0.0605 - accuracy: 0.9893
Epoch 2/5
12/12 [==============================] - 16s 1s/step - loss: 0.0531 - accuracy: 0.9940
Epoch 3/5
12/12 [==============================] - 16s 1s/step - loss: 0.0466 - accuracy: 0.9953
Epoch 4/5
12/12 [==============================] - 16s 1s/step - loss: 0.0414 - accuracy: 0.9957
Epoch 5/5
4/4 [==============================] - 1s 270ms/step - loss: 0.1676 - accuracy: 0.9467


100%|██████████| 30/30 [00:03<00:00,  8.41it/s]


Epoch 1/5
12/12 [==============================] - 16s 1s/step - loss: 0.0404 - accuracy: 0.9933
Epoch 2/5
12/12 [==============================] - 16s 1s/step - loss: 0.0379 - accuracy: 0.9950
Epoch 3/5
12/12 [==============================] - 16s 1s/step - loss: 0.0354 - accuracy: 0.9950
Epoch 4/5
12/12 [==============================] - 16s 1s/step - loss: 0.0336 - accuracy: 0.9953
Epoch 5/5
4/4 [==============================] - 1s 272ms/step - loss: 0.2974 - accuracy: 0.9089
Accuracy - None::  0.33222222328186035
Accuracy - jitter::  0.35777777433395386
Accuracy - scaling::  0.6122221946716309
Accuracy - rotation::  0.9477777481079102
Accuracy - permutation::  0.70333331823349
Accuracy - magnitude_warp::  0.5688889026641846
Accuracy - time_warp::  0.945555567741394
Accuracy - window_slice::  0.9655555486679077
Accuracy - window_warp::  0.949999988079071
Accuracy - spawner::  0.597777783870697
Accuracy - wdba::  0.9911110997200012
Accuracy - random_guided_warp::  0.9466666579246521

**resnet**

In [ ]:
model = get_model("resnet", input_shape, nb_class)
model.compile(optimizer=optm, loss='categorical_crossentropy', metrics=['accuracy'])
method_apply_deep(model, x_train, y_train, x_test, y_test)

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 128, 1)]     0           []                               
                                                                                                  
 conv1d_3 (Conv1D)              (None, 128, 64)      576         ['input_3[0][0]']                
                                                                                                  
 batch_normalization_3 (BatchNo  (None, 128, 64)     256         ['conv1d_3[0][0]']               
 rmalization)                                                                                     
                                                                                                  
 activation_3 (Activation)      (None, 128, 64)      0           ['batch_normalization_3[0][

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


Epoch 1/5
12/12 [==============================] - 34s 3s/step - loss: 0.0468 - accuracy: 0.9970
Epoch 2/5
12/12 [==============================] - 34s 3s/step - loss: 0.0149 - accuracy: 1.0000
Epoch 3/5
12/12 [==============================] - 34s 3s/step - loss: 0.0092 - accuracy: 1.0000
Epoch 4/5
12/12 [==============================] - 34s 3s/step - loss: 0.0072 - accuracy: 1.0000
Epoch 5/5
4/4 [==============================] - 2s 539ms/step - loss: 2.6388 - accuracy: 0.3333
Epoch 1/5
12/12 [==============================] - 35s 3s/step - loss: 0.0501 - accuracy: 0.9867
Epoch 2/5
12/12 [==============================] - 34s 3s/step - loss: 0.0246 - accuracy: 0.9963
Epoch 3/5
12/12 [==============================] - 35s 3s/step - loss: 0.0155 - accuracy: 0.9997
Epoch 4/5
12/12 [==============================] - 34s 3s/step - loss: 0.0106 - accuracy: 0.9997
Epoch 5/5
4/4 [==============================] - 2s 537ms/step - loss: 2.3571 - accuracy: 0.3333
Epoch 1/5
12/12 [=============

100%|██████████| 30/30 [00:00<00:00, 98.65it/s] 


Epoch 1/5
12/12 [==============================] - 34s 3s/step - loss: 0.3739 - accuracy: 0.8557
Epoch 2/5
12/12 [==============================] - 34s 3s/step - loss: 0.2374 - accuracy: 0.8967
Epoch 3/5
12/12 [==============================] - 34s 3s/step - loss: 0.2096 - accuracy: 0.9107
Epoch 4/5
12/12 [==============================] - 34s 3s/step - loss: 0.1791 - accuracy: 0.9260
Epoch 5/5
4/4 [==============================] - 2s 536ms/step - loss: 1.3098 - accuracy: 0.5933


...

Epoch 1/5
12/12 [==============================] - 35s 3s/step - loss: 0.0039 - accuracy: 1.0000
Epoch 2/5
12/12 [==============================] - 34s 3s/step - loss: 0.0022 - accuracy: 1.0000
Epoch 3/5
12/12 [==============================] - 34s 3s/step - loss: 0.0013 - accuracy: 1.0000
Epoch 4/5
12/12 [==============================] - 35s 3s/step - loss: 0.0010 - accuracy: 1.0000
Epoch 5/5
4/4 [==============================] - 2s 540ms/step - loss: 0.1165 - accuracy: 0.9767


100%|██████████| 30/30 [00:00<00:00, 98.55it/s] 


Epoch 1/5
12/12 [==============================] - 35s 3s/step - loss: 0.0157 - accuracy: 0.9990
Epoch 2/5
12/12 [==============================] - 35s 3s/step - loss: 0.0084 - accuracy: 1.0000
Epoch 3/5
12/12 [==============================] - 34s 3s/step - loss: 0.0061 - accuracy: 1.0000
Epoch 4/5
12/12 [==============================] - 34s 3s/step - loss: 0.0047 - accuracy: 1.0000
Epoch 5/5
4/4 [==============================] - 2s 552ms/step - loss: 0.0518 - accuracy: 0.9889


100%|██████████| 30/30 [00:03<00:00,  8.05it/s]


Epoch 1/5
12/12 [==============================] - 34s 3s/step - loss: 0.0066 - accuracy: 1.0000
Epoch 2/5
12/12 [==============================] - 34s 3s/step - loss: 0.0055 - accuracy: 1.0000
Epoch 3/5
12/12 [==============================] - 34s 3s/step - loss: 0.0046 - accuracy: 1.0000
Epoch 4/5
12/12 [==============================] - 34s 3s/step - loss: 0.0042 - accuracy: 1.0000
Epoch 5/5
4/4 [==============================] - 2s 536ms/step - loss: 0.1566 - accuracy: 0.9556
Accuracy - None::  0.34555554389953613
Accuracy - jitter::  0.3311111032962799
Accuracy - scaling::  0.3311111032962799
Accuracy - rotation::  0.3333333432674408
Accuracy - permutation::  0.3333333432674408
Accuracy - magnitude_warp::  0.3333333432674408
Accuracy - time_warp::  0.3333333432674408
Accuracy - window_slice::  0.378888875246048
Accuracy - window_warp::  0.605555534362793
Accuracy - spawner::  0.5933333039283752
Accuracy - wdba::  0.9766666889190674
Accuracy - random_guided_warp::  0.98888885974884

**mlp**

In [ ]:
model = get_model("mlp", input_shape, nb_class)
model.compile(optimizer=optm, loss='categorical_crossentropy', metrics=['accuracy'])
method_apply_deep(model, x_train, y_train, x_test, y_test)

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 128, 1)]          0         
                                                                 
 flatten (Flatten)           (None, 128)               0         
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_3 (Dense)             (None, 500)               64500     
                                                                 
 dropout_2 (Dropout)         (None, 500)               0         
                                                                 
 dense_4 (Dense)             (None, 500)               250500    
                                                                 
 dropout_3 (Dropout)         (None, 500)               0   

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


12/12 [==============================] - 0s 29ms/step - loss: 1.0322 - accuracy: 0.6337
Epoch 2/5
12/12 [==============================] - 0s 29ms/step - loss: 0.7672 - accuracy: 0.6443
Epoch 3/5
12/12 [==============================] - 0s 30ms/step - loss: 0.6866 - accuracy: 0.6830
Epoch 4/5
12/12 [==============================] - 0s 28ms/step - loss: 0.6535 - accuracy: 0.6980
Epoch 5/5
4/4 [==============================] - 0s 8ms/step - loss: 0.3755 - accuracy: 0.8800
Epoch 1/5
12/12 [==============================] - 0s 30ms/step - loss: 0.1303 - accuracy: 0.9950
Epoch 2/5
12/12 [==============================] - 0s 29ms/step - loss: 0.0723 - accuracy: 0.9977
Epoch 3/5
12/12 [==============================] - 0s 28ms/step - loss: 0.0479 - accuracy: 0.9967
Epoch 4/5
12/12 [==============================] - 0s 29ms/step - loss: 0.0363 - accuracy: 0.9980
Epoch 5/5
4/4 [==============================] - 0s 8ms/step - loss: 0.2435 - accuracy: 0.8922
Epoch 1/5
12/12 [===================

100%|██████████| 30/30 [00:00<00:00, 80.72it/s]


Epoch 1/5
12/12 [==============================] - 0s 29ms/step - loss: 0.6772 - accuracy: 0.7877
Epoch 2/5
12/12 [==============================] - 0s 31ms/step - loss: 0.5017 - accuracy: 0.8033
Epoch 3/5
12/12 [==============================] - 0s 30ms/step - loss: 0.4102 - accuracy: 0.8270
Epoch 4/5
12/12 [==============================] - 0s 29ms/step - loss: 0.3789 - accuracy: 0.8447
Epoch 5/5
4/4 [==============================] - 0s 8ms/step - loss: 0.1757 - accuracy: 0.9478


...

100%|██████████| 30/30 [00:00<00:00, 97.99it/s]

Epoch 1/5
 1/12 [=>............................] - ETA: 0s - loss: 0.1543 - accuracy: 0.9375

12/12 [==============================] - 0s 31ms/step - loss: 0.1860 - accuracy: 0.9277
Epoch 2/5
12/12 [==============================] - 0s 30ms/step - loss: 0.1654 - accuracy: 0.9350
Epoch 3/5
12/12 [==============================] - 0s 30ms/step - loss: 0.1518 - accuracy: 0.9473
Epoch 4/5
12/12 [==============================] - 0s 31ms/step - loss: 0.1294 - accuracy: 0.9580
Epoch 5/5
4/4 [==============================] - 0s 9ms/step - loss: 0.1304 - accuracy: 0.9456


100%|██████████| 30/30 [00:03<00:00,  7.75it/s]


Epoch 1/5
12/12 [==============================] - 0s 30ms/step - loss: 0.1167 - accuracy: 0.9583
Epoch 2/5
12/12 [==============================] - 0s 30ms/step - loss: 0.1142 - accuracy: 0.9610
Epoch 3/5
12/12 [==============================] - 0s 31ms/step - loss: 0.1050 - accuracy: 0.9623
Epoch 4/5
12/12 [==============================] - 0s 30ms/step - loss: 0.1082 - accuracy: 0.9617
Epoch 5/5
4/4 [==============================] - 0s 9ms/step - loss: 0.1218 - accuracy: 0.9489
Accuracy - None::  0.5
Accuracy - jitter::  0.8966666460037231
Accuracy - scaling::  0.9188888669013977
Accuracy - rotation::  0.8688889145851135
Accuracy - permutation::  0.8799999952316284
Accuracy - magnitude_warp::  0.8922222256660461
Accuracy - time_warp::  0.8888888955116272
Accuracy - window_slice::  0.9577777981758118
Accuracy - window_warp::  0.945555567741394
Accuracy - spawner::  0.9477777481079102
Accuracy - wdba::  0.9277777671813965
Accuracy - random_guided_warp::  0.945555567741394
Accuracy - 

**lenet**

In [ ]:
model = get_model("lenet", input_shape, nb_class)
model.compile(optimizer=optm, loss='categorical_crossentropy', metrics=['accuracy'])
method_apply_deep(model, x_train, y_train, x_test, y_test)

pooling layers: 4
Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 128, 1)]          0         
                                                                 
 conv1d_14 (Conv1D)          (None, 128, 6)            24        
                                                                 
 max_pooling1d (MaxPooling1D  (None, 64, 6)            0         
 )                                                               
                                                                 
 conv1d_15 (Conv1D)          (None, 64, 16)            304       
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 32, 16)           0         
 1D)                                                             
                                                                 
 conv1d_16 (Conv1D)          (None, 32, 2

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


Epoch 1/5
12/12 [==============================] - 0s 34ms/step - loss: 1.0886 - accuracy: 0.5987
Epoch 2/5
12/12 [==============================] - 0s 35ms/step - loss: 0.7954 - accuracy: 0.6267
Epoch 3/5
12/12 [==============================] - 0s 34ms/step - loss: 0.7236 - accuracy: 0.6650
Epoch 4/5
12/12 [==============================] - 0s 34ms/step - loss: 0.6693 - accuracy: 0.7073
Epoch 5/5
4/4 [==============================] - 0s 13ms/step - loss: 0.3754 - accuracy: 0.8811
Epoch 1/5
12/12 [==============================] - 0s 36ms/step - loss: 0.1861 - accuracy: 0.9620
Epoch 2/5
12/12 [==============================] - 0s 35ms/step - loss: 0.1150 - accuracy: 0.9757
Epoch 3/5
12/12 [==============================] - 0s 36ms/step - loss: 0.0827 - accuracy: 0.9787
Epoch 4/5
12/12 [==============================] - 0s 34ms/step - loss: 0.0570 - accuracy: 0.9880
Epoch 5/5
4/4 [==============================] - 0s 13ms/step - loss: 0.3271 - accuracy: 0.9000
Epoch 1/5
12/12 [=======

100%|██████████| 30/30 [00:00<00:00, 92.01it/s]


Epoch 1/5
12/12 [==============================] - 0s 39ms/step - loss: 0.6703 - accuracy: 0.8077
Epoch 2/5
12/12 [==============================] - 0s 39ms/step - loss: 0.4459 - accuracy: 0.8330
Epoch 3/5
12/12 [==============================] - 0s 39ms/step - loss: 0.3873 - accuracy: 0.8477
Epoch 4/5
12/12 [==============================] - 0s 35ms/step - loss: 0.3577 - accuracy: 0.8623
Epoch 5/5
4/4 [==============================] - 0s 14ms/step - loss: 0.1030 - accuracy: 0.9700


...

12/12 [==============================] - 0s 36ms/step - loss: 0.0298 - accuracy: 0.9980
Epoch 2/5
12/12 [==============================] - 0s 34ms/step - loss: 0.0156 - accuracy: 0.9997
Epoch 3/5
12/12 [==============================] - 0s 35ms/step - loss: 0.0124 - accuracy: 0.9987
Epoch 4/5
12/12 [==============================] - 0s 34ms/step - loss: 0.0088 - accuracy: 0.9990
Epoch 5/5
4/4 [==============================] - 0s 13ms/step - loss: 0.0665 - accuracy: 0.9744


100%|██████████| 30/30 [00:00<00:00, 99.08it/s] 


Epoch 1/5
12/12 [==============================] - 0s 33ms/step - loss: 0.0950 - accuracy: 0.9637
Epoch 2/5
12/12 [==============================] - 0s 34ms/step - loss: 0.0724 - accuracy: 0.9790
Epoch 3/5
12/12 [==============================] - 0s 34ms/step - loss: 0.0604 - accuracy: 0.9823
Epoch 4/5
12/12 [==============================] - 0s 33ms/step - loss: 0.0457 - accuracy: 0.9853
Epoch 5/5
4/4 [==============================] - 0s 13ms/step - loss: 0.0601 - accuracy: 0.9767


100%|██████████| 30/30 [00:03<00:00,  8.28it/s]


Epoch 1/5
12/12 [==============================] - 0s 35ms/step - loss: 0.0321 - accuracy: 0.9920
Epoch 2/5
12/12 [==============================] - 0s 34ms/step - loss: 0.0326 - accuracy: 0.9900
Epoch 3/5
12/12 [==============================] - 0s 34ms/step - loss: 0.0316 - accuracy: 0.9923
Epoch 4/5
12/12 [==============================] - 0s 35ms/step - loss: 0.0294 - accuracy: 0.9910
Epoch 5/5
4/4 [==============================] - 0s 13ms/step - loss: 0.0746 - accuracy: 0.9711
Accuracy - None::  0.3633333444595337
Accuracy - jitter::  0.8933333158493042
Accuracy - scaling::  0.898888885974884
Accuracy - rotation::  0.8399999737739563
Accuracy - permutation::  0.8811110854148865
Accuracy - magnitude_warp::  0.8999999761581421
Accuracy - time_warp::  0.9055555462837219
Accuracy - window_slice::  0.9466666579246521
Accuracy - window_warp::  0.949999988079071
Accuracy - spawner::  0.9700000286102295
Accuracy - wdba::  0.9744444489479065
Accuracy - random_guided_warp::  0.976666688919